# Notable Narratives and content

# collective repository
- GitHub:   https://github.com/ModelSEED/MicrobiomeNotebooks.git

# utilized Narratives
- MAG genomes: https://narrative.kbase.us/narrative/155805 & https://narrative.kbase.us/narrative/163264
- Pangenome-enhanced genomes: https://narrative.kbase.us/narrative/187797
- Annotated pangenome-enhanced genomes: https://narrative.kbase.us/narrative/188040
- Reconstructed MAG models: https://narrative.kbase.us/narrative/186678

# previously-computed files
- FAA of representative sequences: /scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome.faa
- Cluster genome mapping: /scratch/fliu/data/cliff/mmseqs_ani_prob.json

In [16]:
genome_ws = [155805, 163264]
mag_ws = 188065
new_workspace = 187471
pangenome_mag_ws = 187040
prob_mapping = "/scratch/fliu/data/cliff/mmseqs_ani_prob_v2.json"
prob_rep_mapping = "/scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome_v2.faa"

# Loading hit data from json file

In [17]:
%run cliffcommutil.py
with open("/scratch/fliu/data/cliff/mmseqs_ani_prob_v2.json", 'r') as f:
    hit_data = json.load(f)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Parsing hits for close genome list and establishing thresholds

In [18]:
%run cliffcommutil.py
mag_ani_data = {}
mag_thresholds = {}
mag_genomes = {}
mag_hit_distribution = {}
max_bins = 27
min_sim = 74
for protein in hit_data:
    for mag in hit_data[protein]:
        if mag not in mag_ani_data:
            mag_ani_data[mag] = {}
            mag_hit_distribution[mag] = [0] * max_bins
        for hit in hit_data[protein][mag]:
            genome = hit.split(":")[0]
            if hit_data[protein][mag][hit] < 100 and hit_data[protein][mag][hit] >= min_sim:
                if genome not in mag_ani_data[mag]:
                    mag_hit_distribution[mag][int(hit_data[protein][mag][hit]-min_sim)] += 1
                mag_ani_data[mag][genome] = hit_data[protein][mag][hit]
util.save("mag_ani_data",mag_ani_data)
util.save("mag_hit_distribution",mag_hit_distribution)
records = []
for mag in mag_hit_distribution:
    if mag not in mag_thresholds:
        mag_thresholds[mag] = {"threshold":0,"threshold_count":0}
    record = {"mag":mag}
    threshold = 0
    for i in range(max_bins):
        record["ani_" + str(min_sim+i)] = mag_hit_distribution[mag][i]
    threshold_count = 0
    for i in range(max_bins):
        index = max_bins-1-i  
        if mag_hit_distribution[mag][index] > 0 and (threshold == 0 or threshold == min_sim+index+1) and threshold_count <= 4 and (threshold_count == 0 or mag_hit_distribution[mag][index]/threshold_count < 5):
            threshold_count += mag_hit_distribution[mag][index]
            threshold = min_sim+index
    mag_thresholds[mag]["threshold"] = threshold
    mag_thresholds[mag]["threshold_count"] = threshold_count
    if threshold_count == 1:
        new_threshold = 0
        additional_count = 0
        start = max_bins-(threshold-min_sim)
        for i in range(start,max_bins,1):
            index = max_bins-1-i
            if mag_hit_distribution[mag][index] > 0 and (new_threshold == 0 or new_threshold == min_sim+index+1) and additional_count <= 4 and (additional_count == 0 or mag_hit_distribution[mag][index]/additional_count < 5):
                additional_count += mag_hit_distribution[mag][index]
                new_threshold = min_sim+index
        if new_threshold != 0:
            record["new_threshold"] = new_threshold
            record["new_threshold_count"] = additional_count
            if (threshold-new_threshold)<=10 or threshold != 100:
                mag_thresholds[mag]["threshold"] = new_threshold
                mag_thresholds[mag]["threshold_count"] = threshold_count+additional_count
    record["threshold"] = threshold
    record["threshold_count"] = threshold_count
    records.append(record)
for mag in mag_ani_data:
    mag_genomes[mag] = {}
    for genome in mag_ani_data[mag]:
        if mag_ani_data[mag][genome] >= mag_thresholds[mag]["threshold"]:
            mag_genomes[mag][genome] = mag_ani_data[mag][genome]
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/mag_hit_distribution.csv",index=False)
util.save("mag_thresholds",mag_thresholds)
util.save("mag_genomes",mag_genomes)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Assigning thresholds manually (not needed)

In [19]:
%run cliffcommutil.py
mag_thresholds = util.load("mag_thresholds")
mag_ani_data = util.load("mag_ani_data")
mag_genomes = {}
for mag in mag_ani_data:
    mag_genomes[mag] = {}
    for genome in mag_ani_data[mag]:
        if mag_ani_data[mag][genome] >= mag_thresholds[mag]["threshold"]:
            mag_genomes[mag][genome] = mag_ani_data[mag][genome]
    mag_thresholds[mag]["threshold_count"] = len(mag_genomes[mag])

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Getting workspace IDs for MAGs

In [20]:
%run cliffcommutil.py
mag_wsids = {}
for ws in genome_ws:
    for mag in util.msrecon.kbase_api.list_objects(ws, object_type="KBaseGenomes.Genome", include_metadata=False):
        mag_wsids[mag[1]] = mag[7]
util.save("mag_wsids",mag_wsids)
util.save("mag_list",mag_list)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Identifying supplemental proteins for MAGs based on thresholds and hits

In [23]:
%run cliffcommutil.py
mag_protein_supplements = {}
mag_thresholds = util.load("mag_thresholds")
mag_genomes = util.load("mag_genomes")
mag_list = util.load("mag_list")
mag_wsids = util.load("mag_wsids")
magNames = util.load("mag_to_name")
all_mag_deep_data = {}
for magname in mag_wsids:
    all_mag_deep_data[magname] = {}
for protein in hit_data:
    for mag in hit_data[protein]:
        magname = magNames[mag]+".pg.G.D"
        if mag_thresholds[mag]["threshold"] > 0:
            nr_mags_found = {}
            rejected_mags_found = {}
            genomes_found = {}
            all_mag_deep_data[magname][protein] = {
                "self":False,
                "total":[0,0],
                "nr_mags":[0,0],
                "rejected_mags":[0,0],
                "genomes":[0,0]
            }
            for hit in hit_data[protein][mag]:
                (genome,gene) = hit.split(":")
                if genome == "self":
                    all_mag_deep_data[magname][protein]["self"] = True
                elif genome in mag_list:
                    if genome not in nr_mags_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["nr_mags"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["nr_mags"][1] += 1
                        nr_mags_found[genome] = hit_data[protein][mag][hit]
                    elif nr_mags_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
                elif genome in mag_wsids:
                    if genome not in rejected_mags_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["rejected_mags"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["rejected_mags"][1] += 1
                        rejected_mags_found[genome] = hit_data[protein][mag][hit]
                    elif rejected_mags_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
                else:
                    if genome not in genomes_found:
                        all_mag_deep_data[magname][protein]["total"][0] += 1
                        all_mag_deep_data[magname][protein]["genomes"][0] += 1
                        if hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]:
                            if genome not in mag_genomes[mag]:
                                print(genome+":"+mag+":"+str(hit_data[protein][mag][hit])+"/"+str(mag_thresholds[mag]["threshold"]))
                            all_mag_deep_data[magname][protein]["total"][1] += 1
                            all_mag_deep_data[magname][protein]["genomes"][1] += 1
                        genomes_found[genome] = hit_data[protein][mag][hit]
                    elif genomes_found[genome] != hit_data[protein][mag][hit]:
                        print("Warning: different similarity values for the same genome")
            #We only add supplemental proteins for a family that does not already have a protein in the MAG
            if not all_mag_deep_data[magname][protein]["self"]:
                if mag not in mag_protein_supplements:
                    mag_protein_supplements[mag] = {}
                mag_protein_supplements[mag][protein] = [all_mag_deep_data[magname][protein]["total"][1],all_mag_deep_data[magname][protein]["nr_mags"][1],all_mag_deep_data[magname][protein]["rejected_mags"][1]]
for magname in all_mag_deep_data:
    util.save("deepdata/"+mag,all_mag_deep_data[magname])
util.save("mag_protein_supplements",mag_protein_supplements)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


KeyboardInterrupt: 

# Load functional data (not needed)

In [8]:
%run cliffcommutil.py
function_hit_data = json.load(open('/home/fliu/cliff_mags/data/annotation_ani_prob_lo_85.json'))
mag_function_supplements = {}
mag_thresholds = util.load("mag_thresholds")
mag_list = util.load("mag_list")
mag_wsids = util.load("mag_wsids")
all_func_mag_deep_data = {}
for mag in mag_wsids:
    all_func_mag_deep_data[mag] = {}
for protein in function_hit_data:
    for mag in function_hit_data[protein]:
        nr_mags_found = {}
        rejected_mags_found = {}
        genomes_found = {}
        all_func_mag_deep_data[mag][protein] = {
            "self":False,
            "total":[0,0],
            "nr_mags":[0,0],
            "rejected_mags":[0,0],
            "genomes":[0,0]
        }
        for hit in function_hit_data[protein][mag]:
            (genome,gene) = hit.split(":")
            if genome == "self":
                all_func_mag_deep_data[mag][protein]["self"] = True
            elif genome in mag_list:
                if genome not in nr_mags_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["nr_mags"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["nr_mags"][1] += 1
                    nr_mags_found[genome] = function_hit_data[protein][mag][hit]
                elif nr_mags_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
            elif genome in mag_wsids:
                if genome not in rejected_mags_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["rejected_mags"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["rejected_mags"][1] += 1
                    rejected_mags_found[genome] = function_hit_data[protein][mag][hit]
                elif rejected_mags_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
            else:
                if genome not in genomes_found:
                    all_func_mag_deep_data[mag][protein]["total"][0] += 1
                    all_func_mag_deep_data[mag][protein]["genomes"][0] += 1
                    if function_hit_data[protein][mag][hit] >= mag_thresholds[mag]["threshold"]/100:
                        all_func_mag_deep_data[mag][protein]["total"][1] += 1
                        all_func_mag_deep_data[mag][protein]["genomes"][1] += 1
                    genomes_found[genome] = function_hit_data[protein][mag][hit]
                elif genomes_found[genome] != function_hit_data[protein][mag][hit]:
                    print("Warning: different similarity values for the same genome")
        #We only add supplemental proteins for a family that does not already have a protein in the MAG
        if not all_func_mag_deep_data[mag][protein]["self"]:
            if mag not in mag_function_supplements:
                mag_function_supplements[mag] = {}
            mag_function_supplements[mag][protein] = [all_func_mag_deep_data[mag][protein]["total"][1],all_func_mag_deep_data[mag][protein]["nr_mags"][1],all_func_mag_deep_data[mag][protein]["rejected_mags"][1]]
for mag in mag_function_supplements:
    util.save("funcdeepdata/"+mag,all_func_mag_deep_data[mag])
util.save("mag_function_supplements",mag_function_supplements)

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/


# Computing stats on additional proteins

In [27]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_thresholds = util.load("mag_thresholds")
#mag_protein_supplements = util.load("mag_protein_supplements")
records = []
mag_probability_threshold = {}
for mag in mag_list:
    if mag[1] in mag_protein_supplements:
        count = mag_thresholds[mag[1]]["threshold_count"]
        record = {"mag":mag[1]}
        records.append(record)
        abundance_count = [0] * 400
        for protein in mag_protein_supplements[mag[1]]:
            fraction = mag_protein_supplements[mag[1]][protein][0]/count
            entry = int(fraction*100/5)
            abundance_count[entry] += 1
        total = 0
        final_total = None
        mag_probability_threshold[mag[1]] = 0.1
        for (i,entry) in enumerate(abundance_count):
            index = len(abundance_count)-i-1
            lasttotal = total
            total += abundance_count[index]
            if total >= 20000 and final_total == None:
                threshold = 5*(index+1)/100
                if threshold >= 0.1:
                    mag_probability_threshold[mag[1]] = threshold
                    final_total = lasttotal
            if index == 2 and final_total == None:
                final_total = total
        print(mag[1],mag_probability_threshold[mag[1]],final_total)
        for (i,entry) in enumerate(abundance_count):
            record[i] = entry
util.save("mag_probability_threshold",mag_probability_threshold)
df = pd.DataFrame.from_records(records)
df.to_csv(util.output_dir+"/protein_count_abundance.csv",index=False)

Salt_Pond_MetaGSF2_B_H2O_MG_DASTool_bins_metabat.8.contigs__.RAST 0.1 14737
Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_metabat.20.contigs__.RAST 0.1 3860
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_concoct_out.4.contigs__.RAST 0.1 4004
Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.21.contigs__.RAST 0.1 4192
Salt_Pond_MetaG_R1_C_D1_MG_DASTool_bins_metabat.31.contigs__.RAST 0.1 5106
Salt_Pond_MetaG_R2_A_H2O_MG_DASTool_bins_concoct_out.29.contigs__.RAST 0.1 3670
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_metabat.50.contigs__.RAST 0.1 4271
Salt_Pond_MetaG_R1_B_D2_MG_DASTool_bins_metabat.18.contigs__.RAST 0.1 4144
Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.15.contigs__.RAST 0.1 1457
Salt_Pond_MetaG_R2_C_D1_MG_DASTool_bins_concoct_out.88.contigs__.RAST 0.1 2330
Salt_Pond_MetaG_R2_restored_C_black_MG_DASTool_bins_concoct_out.57.contigs__.RAST 0.1 1738
Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.10.contigs__.RAST 0.1 2713
Salt_Pond_MetaGSF2_B_D1_MG_DASTool_bins_concoct_out.44.contigs__.RA

# Renaming MAGs

In [3]:
%run cliffcommutil.py
mag_to_name = {}
name_to_mag = {}
mag_list = util.load("mag_list")
for mag in mag_list:
    original_name = mag[1]
    new_name = original_name
    new_name = new_name.replace("Salt_Pond_MetaG_", "", 1)
    new_name = new_name.replace("Salt_Pond_MetaG", "", 1)
    new_name = new_name.replace("_MG_DASTool_bins_", "", 1)
    new_name = new_name.replace("metabat", "", 1)
    new_name = new_name.replace("concoct_out", "", 1)
    new_name = new_name.replace("maxbin", "", 1)
    new_name = new_name.replace(".contigs__.RAST", "", 1)
    mag_to_name[mag[1]] = new_name
    name_to_mag[new_name] = mag[1]
    print(name_to_mag)
util.save("mag_to_name",mag_to_name)
util.save("name_to_mag",name_to_mag)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir


1722057900.422139 ERROR: Requested data mag_list doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/mag_list.json


ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


ValueError: Requested data mag_list doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/mag_list.json

# Loading protein sequence data

In [14]:
%run cliffcommutil.py
from Bio import SeqIO
protein_hash = {}
for record in SeqIO.parse('/scratch/fliu/data/cliff/mmseqs_ani_prob_rep_genome_v2.faa', 'fasta'):
    protein_hash[record.id] = record.seq

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/


# Building genomes and assemblies

In [28]:
%run cliffcommutil.py
unfiltered_list = util.load("mag_list")
filter = [
  "Salt_Pond_MetaG_R2_restored_C_black_MG_DASTool_bins_concoct_out.17.contigs__.RAST",
  "Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_concoct_out.52.contigs__.RAST",
  "Salt_Pond_MetaG_R2_restored_DShore_MG_DASTool_bins_concoct_out.38.contigs__.RAST",
  "Salt_Pond_MetaG_R2A_C_D2_MG_DASTool_bins_metabat.27.contigs__.RAST",
  "Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.25.contigs__.RAST",
  "Salt_Pond_MetaG_R2_B_D1_MG_DASTool_bins_concoct_out.110.contigs__.RAST",
  "Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.47.contigs__.RAST",
  "Salt_Pond_MetaG_R2A_A_D2_MG_DASTool_bins_concoct_out.10.contigs__.RAST",
  "Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.55.contigs__.RAST",
  "Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.32.contigs__.RAST"
]
mag_list = []
for item in unfiltered_list:
    if item[1] in filter or len(filter) == 0:
        mag_list.append(item)
name_to_mag = util.load("name_to_mag")
mag_thresholds = util.load("mag_thresholds")
mag_probability_threshold = util.load("mag_probability_threshold")
#mag_protein_supplements = util.load("mag_protein_supplements")
for item in mag_list:
#for name in problem_list:
    full_data = util.get_object(item[1],item[7])
    genome_obj = full_data["data"]
    genome_obj["assembly_ref"] = str(item[6])+"/"+str(item[0])+"/"+str(item[4])+";"+genome_obj["assembly_ref"]
    name = mag_to_name[item[1]]
    cds_hash = {}
    for i,ftr in enumerate(genome_obj["cdss"]):
        cds_hash[ftr["id"]] = ftr
    ftr_hash = {}
    for i,ftr in enumerate(genome_obj["features"]):
        ftr_hash[ftr["id"]] = ftr
        ftr["id"] = name+"_"+str(i+1)
        if "cdss" in ftr:
            for j,cds in enumerate(ftr["cdss"]):
                cds_hash[cds]["id"] = ftr["id"]+".CDS"
                ftr["cdss"][j] = cds_hash[cds]["id"]
                cds_hash[cds]["parent_gene"] = ftr["id"]
    firstgene = genome_obj["features"][0]
    count = 0
    if item[1] in mag_protein_supplements:
        count = 1
        total_genomes = mag_thresholds[item[1]]["threshold_count"]
        for protein in mag_protein_supplements[item[1]]:
            if mag_protein_supplements[item[1]][protein][0]/total_genomes >= mag_probability_threshold[item[1]]:
                ftrid = name+'.pg_'+str(count)
                count += 1
                protseq = str(protein_hash[protein])
                dnaseq = util.translate_protein_to_gene(protseq)
                result = hashlib.md5(protseq.encode())
                md5 = result.hexdigest()
                result = hashlib.md5(dnaseq.encode())
                dnamd5 = result.hexdigest()
                newftr = {
                    "aliases": [["MMseqMD5",protein]],
                    "cdss": [
                        ftrid+".CDS"
                    ],
                    "functions":["Hypothetical protein"],
                    "dna_sequence": dnaseq,
                    "dna_sequence_length": 3*len(protseq),
                    "id": ftrid,
                    "location": [
                        [
                            firstgene["location"][0][0],
                            1,
                            "+",
                            3*len(protseq)
                        ]
                    ],
                    "md5": dnamd5,
                    "ontology_terms": {},
                    "protein_md5": md5,
                    "protein_translation": protseq,
                    "protein_translation_length": len(protseq),
                    "warnings": []
                }
                cdsftr = newftr.copy()
                del cdsftr["cdss"]
                cdsftr["id"] = ftrid+".CDS"
                cdsftr["parent_gene"] = ftrid
                genome_obj["features"].append(newftr)
                genome_obj["cdss"].append(cdsftr)
        print(item[1],count)
        #Saving MAG
        util.save("genome/"+name,full_data)

Output files printed to:/scratch/chenry/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/kb_sdk_home/run_local/workdir/tmp/
Salt_Pond_MetaGSF2_B_H2O_MG_DASTool_bins_metabat.8.contigs__.RAST 14738
Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_metabat.20.contigs__.RAST 3861
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_concoct_out.4.contigs__.RAST 4005
Salt_Pond_MetaG_R2_restored_DShore_MG_DASTool_bins_concoct_out.46.contigs__.RAST 4005
Salt_Pond_MetaG_R1_A_D2_MG_DASTool_bins_concoct_out.21.contigs__.RAST 4193
Salt_Pond_MetaG_R1_C_D1_MG_DASTool_bins_metabat.31.contigs__.RAST 5107
Salt_Pond_MetaG_R2_A_H2O_MG_DASTool_bins_concoct_out.29.contigs__.RAST 3671
Salt_Pond_MetaG_R2_B_D2_MG_DASTool_bins_metabat.50.contigs__.RAST 4272
Salt_Pond_MetaG_R1_B_D2_MG_DASTool_bins_metabat.18.contigs__.RAST 4145
Salt_Pond_MetaGSF2_C_D2_MG_DASTool_bins_concoct_out.32.contigs__.RAST 4145
Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.15.contigs__.RAST 1458
Salt_Po

# Finding and fixing redundant names

In [ ]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
name_to_mag = {}
for item in mag_list:
    if item[1] not in mag_to_name:
        print("Missing:",item[1])
    elif mag_to_name[item[1]] in name_to_mag:
        print("Collision:",mag_to_name[item[1]])
    else:
        name_to_mag[mag_to_name[item[1]]] = item[1]
util.save("name_to_mag",name_to_mag)

# Loading genomes to KBase

In [ ]:
%run cliffcommutil.py
unfiltered_list = util.load("mag_list")
filter = []
mag_list = []
for item in unfiltered_list:
    if item[1] in filter or len(filter) == 0:
        mag_list.append(item)
mag_to_name = util.load("mag_to_name")
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
workspace = 187797
anno = util.anno_client()
anno.clients["GenomeFileUtil"] = util.gfu_client()
finished = util.load("finished_genomes",[])
unconverted = {}
for item in mag_list:
    name = mag_to_name[item[1]]
    if name not in finished:
        genome = util.load("annotated/"+name)
        count = 0
        for ftr in genome["features"]:
            end = len(name)+3
            if ftr["id"][0:4] == "Salt":
                unconverted[item[1]] = 1
            if ftr["id"][0:end] == name+".pg":
                count += 1
        print(name,count)
        util.save_ws_object(name+".pg",187797,genome,"KBaseGenomes.Genome")
        finished.append(name)
        util.save("finished_genomes",finished)
for item in unconverted:
    print(item)

# Inspecting genomes before load

In [ ]:
%run cliffcommutil.py
mags = util.msrecon.kbase_api.list_objects(187875, object_type="KBaseGenomes.Genome", include_metadata=False)
mag_hash = {}
for item in mags:
    mag_hash[item[1]] = item
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
for item in mag_list:
    name = mag_to_name[item[1]]
    name += ".pangenome.GLM4EC"
    if name in mag_hash:
        print("Getting "+name)
        full_data = util.get_object(name,187875)
        genome = full_data["data"]
        ftrhash = {}
        for ftr in genome["features"]:
            ftrhash[ftr["id"]] = ftr
        for ftr in genome["cdss"]:
            ftrhash[ftr["id"]] = ftr
            if "parent_gene" in ftr:
                if ftr["parent_gene"] not in ftrhash:
                    print(item[1],new_name,"Gene "+ftr["parent_gene"]+" not found!")
        for ftr in genome["features"]:
            if "cdss" in ftr:
                for cds in ftr["cdss"]:
                    if cds not in ftrhash:
                        print(item[1],new_name,"CDS "+cds+" not found!")

# Annotating genomes using RAST API

In [6]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
from modelseedpy.core.rpcclient import RPCClient
client = RPCClient("https://tutorial.theseed.org/services/genome_annotation")
for item in mag_list:
    name = mag_to_name[item[1]]
    if name not in output:
        rast_genome = {
            "id":item[1],
            "genetic_code":11,
            "scientific_name":"Unknown",
            "domain":"Bacteria",
            "contigs": [],
            "features":[]
        }
        data = util.load("genome/"+name)
        genome = data["data"]
        for ftr in genome["features"]:
            rast_genome["features"].append(ftr)
        workflow = {
            "stages":[
                {
                    "name": "annotate_proteins_kmer_v2", 
                    "kmer_v2_parameters": {}},
                {
                    "name": "annotate_proteins_kmer_v1",
                    "kmer_v1_parameters": {"annotate_hypothetical_only": 1},},
                {
                    "name": "annotate_proteins_similarity",
                    "similarity_parameters": {"annotate_hypothetical_only": 1},
                },
            ]
        }
        params = [{"features": rast_genome["features"]}, workflow]
        output = client.call("GenomeAnnotation.run_pipeline",params)
        util.save("rast/"+name,output)

Output files printed to:/Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /Users/chenry/code//kb_sdk/run_local/workdir/tmp/


1722232442.262796 ERROR: Requested data genome/R1_B_D1.20 doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/genome/R1_B_D1.20.json


<Response [200]>


ValueError: Requested data genome/R1_B_D1.20 doesn't exist at /Users/chenry/code/notebooks/MicrobiomeNotebooks/Cliff/datacache/genome/R1_B_D1.20.json

# Adding RAST ontology event

In [ ]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
for item in mag_list:
    name = mag_to_name[item[1]]
    data = util.load("genome/"+name)
    rast = util.load("rast/"+name)
    annoapi = util.anno_client(native_python_api=True)
    events = [{
        "ontology_id" : "SSO",
        "description" : "RAST Annotation API",
        "method_version" : "1.0",
        "method" : "annotate_genome",
        "ontology_terms": {},
        "timestamp":"2024-07-29T19:32:45"
    }]
    for ftr in rast[0]["features"]:
        if "function" in ftr:
            events[0]["ontology_terms"][ftr["id"]] = [{"term":ftr["function"]}]
    output = annoapi.add_annotation_ontology_events({
        "output_workspace":187797,
        "events":events,
        "overwrite_matching":True,
        "object":data["data"],
        "type":"KBaseGenomes.Genome",
        "input_ref":None,
        "input_workspace":None,
        "output_name":None,
        "save":0
    })
    util.save("annotated/"+name,output["object"])

# Printing feature probabilities

In [10]:
%run cliffcommutil.py
mag_list = util.load("mag_list")
mag_thresholds = util.load("mag_thresholds")
mag_to_name = util.load("mag_to_name")
mag_protein_supplements = util.load("mag_protein_supplements")
feature_probabilities = {}
for item in mag_list:
    name = mag_to_name[item[1]]
    data = util.load("annotated/"+name)
    feature_probabilities[item[1]] = {}
    ftrs = data["features"]
    total_genomes = mag_thresholds[item[1]]["threshold_count"]
    for ftr in ftrs:
        if ftr["id"].split(".")[-1][0:3] == "pg_":
            feature_probabilities[item[1]][ftr["id"]] = mag_protein_supplements[item[1]][ftr["aliases"][0][1]][0]/total_genomes
            if feature_probabilities[item[1]][ftr["id"]] > 1:
                feature_probabilities[item[1]][ftr["id"]] = 1
        else:
            feature_probabilities[item[1]][ftr["id"]] = 1
util.save("feature_probabilities",feature_probabilities)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Processing metabolite data

In [3]:
%run cliffcommutil.py
metabolite_hash = util.load("../data/met_NameMSID")

from numpy import array
newMets = array(["2'-Deoxyuridine", '2-Oxoglutarate', '2-Oxoisocaproate', '3-Hydroxybutyrate', '3-Hydroxyisovalerate', '3-Methyl-2-oxovalerate', '4-Aminobutyrate', 'Acetate', 'Acetone', 'Alanine', 'Arginine', 'Aspartate', 'Benzoate', 'Betaine', 'Dimethylamine', 'Ethanol', 'Formate', 'Fructose', 'Fumarate', 'Glucose', 'Glutamate', 'Glycerol', 'Isobutyrate', 'Isoleucine', 'Isopropanol', 'Isovalerate', 'Lactate', 'Leucine', 'Maltose', 'Methanol', 'Methionine', 'Methylamine', 'Methylguanidine', 'N,N-Dimethylglycine', 'Phenylacetate', 'Phenylalanine', 'Proline', 'Propionate', 'Propylene glycol', 'Pyroglutamate', 'Succinate', 'Sucrose', 'Thymidine', 'Trehalose', 'Trimethylamine', 'Tryptophan', 'Tyrosine', 'Uracil', 'Uridine', 'Valine'])
print(set(newMets)-set(list(metabolite_hash.keys())))

metabolite_names = {cpdID: metName for metName, cpdID in metabolite_hash.items()}
util.save("metabolite_names",metabolite_names)
util.save("metabolites",list(metabolite_names.keys()))

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
set()


# Computing SMIPPs

In [1]:
%run cliffcommutil.py

def smipps(arguments):
    # unpack the arguments
    (mag_list, mags_to_models, model_ws, metabolites, feature_probabilities, reaction_probabilities, problemlist, auxo_media, gmm_base_media, aerobicity
    ) = arguments
    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    # define the phenotype sets
    uptake_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=auxo_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="uptake"
    )
    excretion_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=auxo_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="excretion"
    )
    growth_phenoset = util.create_phenotypeset_from_compounds(
        metabolites,
        base_media=gmm_base_media,
        base_uptake=0,
        base_excretion=1000,
        global_atom_limits={},
        type="growth"
    )
    phenosets = {"uptake":uptake_phenoset,"excretion":excretion_phenoset,"growth":growth_phenoset}
    
    # compute the SMIPP for each MAG
    gf_phenotype_results = {}
    for i,mag in enumerate(mag_list):
        ## load the mode
        print(mag)
        name = mag[1]
        if name in problemlist:  continue
        try:
            mdlutl = util.msrecon.get_model(name+model_suffix,model_ws)
        except Exception as e:
            print(f"The {name+model_suffix} object is not in the {model_ws} worksheet")
            print(e)
            
        ## determine the highest probability for each gene associated with a reaction in the model
        reaction_probabilities[name] = {}
        for rxn in mdlutl.model.reactions:
            highest_prob = None
            for gene in rxn.genes:
                if gene.id in feature_probabilities[mag[1]]:
                    if highest_prob == None or feature_probabilities[mag[1]][gene.id] > highest_prob:
                        highest_prob = feature_probabilities[mag[1]][gene.id]
            if highest_prob != None:
                rxn.probability = highest_prob
                reaction_probabilities[name][rxn.id] = highest_prob

        ## assign the highest possible probability among the annotated genes for each reaction
        print(mdlutl.model.genome_ref)
        genome = util.msrecon.get_msgenome_from_ontology(mdlutl.model.genome_ref,native_python_api=True,output_ws=None)
        reaction_hash = genome.annoont.get_reaction_gene_hash(feature_type="gene")
        for rxn in reaction_hash:
            highest_prob = None
            for gene in reaction_hash[rxn]:
                if gene in feature_probabilities[mag[1]]:
                    if highest_prob == None or feature_probabilities[mag[1]][gene] > highest_prob:
                        highest_prob = feature_probabilities[mag[1]][gene]
            if highest_prob != None and (rxn+"_c0" not in reaction_probabilities[name] or highest_prob >= reaction_probabilities[name][rxn+"_c0"]):
                if rxn+"_c0" in mdlutl.model.reactions:
                    mdlutl.model.reactions.get_by_id(rxn+"_c0").probability = highest_prob
                reaction_probabilities[name][rxn+"_c0"] = highest_prob

        ## gapfill the model
        filters = mdlutl.get_attributes("gf_filter")
        tests = mdlutl.get_atp_tests(core_template=util.msrecon.core_template,atp_media_filename=util.msrecon.module_dir+"/data/atp_medias.tsv",recompute=False)
        msgapfill = MSGapfill(
            mdlutl,
            [util.msrecon.get_template(mdlutl.model.template_ref)],
            [],
            tests,
            blacklist=[],
            default_target="bio1",
            minimum_obj=0.01,
            base_media=None,
            base_media_target_element=None
        )

        ### Adding missing transporter for metabolites to gapfilling database
        for cpd in metabolites:
            if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)

        ## create flux coefficients from the gapfilling
        coefficients = {}
        gf_penalties = msgapfill.gfpkgmgr.getpkg("GapfillingPkg").gapfilling_penalties
        gfrxn = 0
        probrxn = 0
        otherrxn = 0
        for reaction in msgapfill.gfmodelutl.model.reactions:
            if reaction.id in reaction_probabilities[name]:
                probrxn += 2
                coefficients[">"+reaction.id] = 1-reaction_probabilities[name][reaction.id]
                coefficients["<"+reaction.id] = 1-reaction_probabilities[name][reaction.id]
            elif reaction.id in gf_penalties:
                if "forward" in gf_penalties[reaction.id]:
                    gfrxn += 1
                    coefficients[">"+reaction.id] = 1+gf_penalties[reaction.id]["forward"]
                else:
                    otherrxn += 1
                    coefficients[">"+reaction.id] = 0.95
                if "reverse" in gf_penalties[reaction.id]:
                    gfrxn += 1
                    coefficients["<"+reaction.id] = 1+gf_penalties[reaction.id]["reverse"]
                else:
                    otherrxn += 1
                    coefficients["<"+reaction.id] = 0.95
            else:
                otherrxn += 2
                coefficients[">"+reaction.id] = 0.95
                coefficients["<"+reaction.id] = 0.95
        print(name,"GF:",gfrxn,"Prob:",probrxn,"Other:",otherrxn)

        #TODO Create conditional logic for growth phenotypes if errors are raised here 
        msgapfill.prefilter(test_conditions=tests,growth_conditions=[],use_prior_filtering=True,base_filter_only=True)

        ## simulate the SMIPP condition and parse the results
        gf_phenotype_results[name] = {}
        for phenoid in phenosets:
            gf_phenotype_results[name][phenoid] = {}
            output = phenosets[phenoid].simulate_phenotypes(
                msgapfill.gfmodelutl,
                multiplier=2,
                add_missing_exchanges=True,
                save_fluxes=False,
                save_reaction_list=True,
                gapfill_negatives=False,
                msgapfill=None,
                test_conditions=None,
                ignore_experimental_data=True,
                flux_coefficients=coefficients
            )
            for index, row in output["details"].iterrows():
                if "reactions" in output["data"][row["Phenotype"]]:
                    output["data"][row["Phenotype"]]["average_probability"] = 0
                    for rxn in output["data"][row["Phenotype"]]["reactions"]:
                        direction = rxn[0:1]
                        rxnid = rxn[1:]
                        if direction == ">":
                            if rxnid not in gf_penalties or "forward" not in gf_penalties[rxnid]:
                                if rxnid in reaction_probabilities[name]:
                                    output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[name][rxnid]
                                else:
                                    output["data"][row["Phenotype"]]["average_probability"] += 0.05
                        elif direction == "<":
                            if rxnid not in gf_penalties or "reverse" not in gf_penalties[rxnid]:
                                if rxnid in reaction_probabilities[name]:
                                    output["data"][row["Phenotype"]]["average_probability"] += reaction_probabilities[name][rxnid]
                                else:
                                    output["data"][row["Phenotype"]]["average_probability"] += 0.05
                    output["data"][row["Phenotype"]]["average_probability"] = output["data"][row["Phenotype"]]["average_probability"]/len(output["data"][row["Phenotype"]]["reactions"])
                gf_phenotype_results[name][phenoid][row["Phenotype"]] = output["data"][row["Phenotype"]]
            
    util.save(f"SMIPPs/new_gf_phenotype_results_{pid}",gf_phenotype_results)
    util.save(f"SMIPPs/reaction_probabilities_{pid}",reaction_probabilities)
                
# define function parameters
mag_list = util.load("mag_list")
model_ws = 188065
model_suffix = ".pyr"
metabolites = util.load("metabolites")
feature_probabilities = util.load("feature_probabilities")
reaction_probabilities = util.load("reaction_probabilities")
problemlist = util.load("problemlist",[])
mag_to_name = util.load("mag_to_name")
for mag in mag_list:
    mag[1] = mag_to_name[mag[1]]+".pg.G.D"
mag_feature_probabilities = {mag_to_name[k]+".pg.G.D":v for k,v in feature_probabilities.items()}

auxo_media = util.msrecon.get_media(f"{model_ws}/AuxoMedia")
gmm_base_media = util.msrecon.get_media(f"{model_ws}/PyruateMinimalAerobic")

# run the parallelized code
parallelize = True
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    mag_lists = array_split(mag_list, int(len(mag_list)/4))
    pool = Pool(min(cpu_count(), len(mag_lists)))
    args = [(mag_list, mag_to_name, model_ws, metabolites, mag_feature_probabilities, reaction_probabilities, problemlist, auxo_media, gmm_base_media, "aerobic")
           for mag_list in mag_lists]
    list_of_outputs = pool.map(smipps, args)
else:
    outputs = smipps((remaining_mags, mag_to_name, model_ws, metabolites, mag_feature_probabilities, reaction_probabilities, problemlist, auxo_media, gmm_base_media, "aerobic"))

python version 3.11.1
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
ForkPoolWorker-1
[3 'SF2_B_H2O.8.pg.G.D' 'KBaseGenomes.Genome-11.1'
 '2023-09-02T02:08:15+0000' 1 'cliff_bueno' 155805
 'cliff_bueno:narrative_1693612235241' 'db630a36d897caba142685e9e010ae30'
 9852570 None]


1723048932.944119 INFO: metabolites 1176
1723048933.4564693 INFO: reactions 1157
1723048933.5926237 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723048934.2600315 INFO: Default biomass: [bio1]


188040/338/1


1723048934.4035861 INFO: get_annotation_ontology_events:{
    "input_ref": "188040/338/1"
}
1723048940.870303 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.
1723048941.5282176 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.


Getting tests from attributes


...

1723053310.9152334 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.
1723053311.3434656 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.


Getting tests from attributes


1723053314.3989382 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723053319.624403 INFO: cpd05178 not found in model!
1723053319.9004157 INFO: cpd05178 not found in model!
1723053319.9091704 INFO: Media compound: cpd05178 not found in model.
1723053327.7071722 INFO: cpd05178 not found in model!
1723053327.7139208 CRITICAL: Metabolite cpd05178 not found in model


R2A_A_D1.24.pg.G.D GF: 11424 Prob: 3954 Other: 2974


1723053338.5670395 INFO: Objective with gapfill database:0.0; min objective:0.01
/opt/env/python3_modelseed/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
1723053352.1110282 INFO: Expansion time:Glc:0.8522181410007761
1723053352.115936 INFO: Filtered count:0 out of 1456
1723053352.9710886 INFO: Expansion time:Glc.DMSO:0.8423251710009936
1723053352.9835794 INFO: Filtered count:0 out of 1456
1723053353.8428197 INFO: Expansion time:Glc.O2:0.8555071569990105
1723053353.8469527 INFO: Filtered count:0 out of 1456
1723053357.1709313 INFO: Expansion time:Glc.TMAO:3.3225352079989534
1723053357.1739745 INFO: Filtered count:0 out of 1456
1723053357.9660146 INFO: Expansion time:Glyc:0.7903525109995826
1723053357.9730146 INFO: Filtered count:0 out of 1456
1723053358.9634717 INFO: Expansion time:Glyc.O2:0.9857639450001443
1723053358.972334 INFO: Filtered count:0 out of 1456
1723053359.9550784 INF

# Combining the outputs from each parallel thread

In [1]:
from json import load, dump
from glob import glob

new_gf_phenotype_results = {}
for path in glob(r"datacache/SMIPPs/new_gf_phenotype_results_*.json"):
    with open(path, 'r') as jsonIn:
        new_gf_phenotype_results.update(load(jsonIn))
        
with open("datacache/SMIPPs/new_gf_phenotype_results.json", 'w') as jsonOut:
    dump(new_gf_phenotype_results, jsonOut, indent=3)

In [2]:
print(len(new_gf_phenotype_results.keys()))

310


In [3]:
from json import load, dump
from glob import glob

reaction_probabilities = {}
for path in glob(r"datacache/SMIPPs/reaction_probabilities_*.json"):
    with open(path, 'r') as jsonIn:
        reaction_probabilities.update(load(jsonIn))
        
with open("datacache/SMIPPs/reaction_probabilities.json", 'w') as jsonOut:
    dump(reaction_probabilities, jsonOut, indent=3)

# Computing proper gapfill and filtering bad predictions

### not penalize existing gapfilled reactions, and emphasizing the probabilites of newly gapfilled reactions

In [1]:
%run cliffcommutil.py


def cleanProbabilities(arguments):
    # load arguments
    (mag_names_list, name_to_mag, studies, consolidated_gf_results, index) = arguments
    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    for name in mag_names_list:
        modelname = name_to_mag[name.replace(".pg.G.D", '')].replace(".contigs__.RAST", ".pangenome.RAST.glm4ec.pyr")
        print(modelname)
        mdlutl = util.msrecon.get_model(modelname,186678)
        for study in studies:
            if study not in consolidated_gf_results[name]:
                continue
            for phenotype in consolidated_gf_results[name][study]:
                if "reactions" in consolidated_gf_results[name][study][phenotype]:
                    consolidated_gf_results[name][study][phenotype]["new_probability"] = 0
                    consolidated_gf_results[name][study][phenotype]["probrxn"] = 0
                    consolidated_gf_results[name][study][phenotype]["otherrxn"] = 0
                    consolidated_gf_results[name][study][phenotype]["origgfrxn"] = 0
                    consolidated_gf_results[name][study][phenotype]["new_gf"] = 0
                    consolidated_gf_results[name][study][phenotype]["new_gf_reactions"] = []
                    found = False
                    for rxn in consolidated_gf_results[name][study][phenotype]["reactions"]:
                        direction = rxn[0:1]
                        rxnid = rxn[1:]
                        if rxnid[0:3] == "EX_" and rxnid[3:11] == phenotype:
                            found = True
                        if rxnid not in mdlutl.model.reactions:
                            consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                            consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                        elif direction == ">":
                            if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                                consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                                consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                            elif rxnid in reaction_probabilities[name]:
                                consolidated_gf_results[name][study][phenotype]["new_probability"] += reaction_probabilities[name][rxnid]
                                consolidated_gf_results[name][study][phenotype]["probrxn"] += 1
                            elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                                consolidated_gf_results[name][study][phenotype]["origgfrxn"] += 1
                            else:
                                consolidated_gf_results[name][study][phenotype]["otherrxn"] += 1
                                consolidated_gf_results[name][study][phenotype]["new_probability"] += 0.05  
                        elif direction == "<":
                            if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                                consolidated_gf_results[name][study][phenotype]["new_gf_reactions"].append(rxn)
                                consolidated_gf_results[name][study][phenotype]["new_gf"] += 1
                            elif rxnid in reaction_probabilities[name]:
                                consolidated_gf_results[name][study][phenotype]["new_probability"] += reaction_probabilities[name][rxnid]
                                consolidated_gf_results[name][study][phenotype]["probrxn"] += 1
                            elif rxnid[0:3] != "bio" and rxnid[0:3] != "EX_" and rxnid[0:3] != "DM_" and len(mdlutl.model.reactions.get_by_id(rxnid).genes) == 0:
                                consolidated_gf_results[name][study][phenotype]["origgfrxn"] += 1
                            else:
                                consolidated_gf_results[name][study][phenotype]["otherrxn"] += 1
                                consolidated_gf_results[name][study][phenotype]["new_probability"] += 0.05  
                    totalrxn = consolidated_gf_results[name][study][phenotype]["probrxn"]+consolidated_gf_results[name][study][phenotype]["new_gf"]+consolidated_gf_results[name][study][phenotype]["otherrxn"]
                    consolidated_gf_results[name][study][phenotype]["new_probability"] = consolidated_gf_results[name][study][phenotype]["new_probability"]/totalrxn

                    if not found:
                        consolidated_gf_results[name][study][phenotype]["original_objective_value"] = consolidated_gf_results[name][study][phenotype]["objective_value"]
                        consolidated_gf_results[name][study][phenotype]["objective_value"] = 0
                        consolidated_gf_results[name][study][phenotype]["class"] = "N"
                        consolidated_gf_results[name][study][phenotype]["positive"] = False
                        consolidated_gf_results[name][study][phenotype]["new_probability"] = 0
    util.save(f"cleaned_SMIPPs/gf_phenotype_results_{index}",consolidated_gf_results)
                    

# define the arguments
consolidated_gf_results = util.load("SMIPPs/new_gf_phenotype_results")
reaction_probabilities = util.load("SMIPPs/reaction_probabilities")
name_to_mag = util.load("name_to_mag")
studies = ["uptake","excretion","growth"]

# run the parallelized code
parallelize = True
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    mag_names_lists = array_split(list(consolidated_gf_results.keys()), int(len(consolidated_gf_results)/4))
    pool_count = min(cpu_count(), len(mag_names_lists))
    print(f"{pool_count} threads will be created")
    pool = Pool(pool_count)
    args = [(mag_names_list, name_to_mag, studies, {k:v for k,v in consolidated_gf_results.items() if k in mag_names_list}, index)
           for index, mag_names_list in enumerate(mag_names_lists)]
    list_of_outputs = pool.map(cleanProbabilities, args)
else:
    outputs = cleanProbabilities([[list(consolidated_gf_results.keys())[0]], name_to_mag, studies, consolidated_gf_results, 0])

python version 3.11.1
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.3.1
Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
77 threads will be created
ForkPoolWorker-1
Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.45.pangenome.RAST.glm4ec.pyr
ForkPoolWorker-2
Salt_Pond_MetaG_R2_C_D1_MG_DASTool_bins_concoct_out.88.pangenome.RAST.glm4ec.pyr
ForkPoolWorker-3
Salt_Pond_MetaG_R1_A_D1_MG_DASTool_bins.metabat.18.pangenome.RAST.glm4ec.pyr


1723055219.2942505 INFO: metabolites 1256


ForkPoolWorker-4
Salt_Pond_MetaG_R2A_B_H2O_MG_DASTool_bins_concoct_out.69.pangenome.RAST.glm4ec.pyr


1723055219.406164 INFO: metabolites 1042


ForkPoolWorker-5
Salt_Pond_MetaGSF2_A_H2O_MG_DASTool_bins_metabat.14.pangenome.RAST.glm4ec.pyr


...

1723055251.1209784 INFO: reactions 1083
1723055251.1242132 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055251.178235 INFO: metabolites 1214
1723055251.2530744 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055251.2654195 INFO: metabolites 1301
1723055251.2721524 INFO: reactions 639
1723055251.4193377 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055251.4996333 INFO: metabolites 1135
1723055251.5222888 INFO: Default biomass: [bio1]
1723055251.5397208 INFO: Default biomass: [bio1]


Salt_Pond_MetaG_R2A_A_D1_MG_DASTool_bins_concoct_out.24.pangenome.RAST.glm4ec.pyr


1723055251.6740706 INFO: Default biomass: [bio1]
1723055251.6845047 INFO: Default biomass: [bio1]


Salt_Pond_MetaG_R2A_C_D2_MG_DASTool_bins_metabat.27.pangenome.RAST.glm4ec.pyr


1723055252.0118968 INFO: metabolites 1040
1723055252.1369624 INFO: metabolites 1110
1723055252.1476576 INFO: reactions 1080
1723055252.277363 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055252.6508863 INFO: reactions 1197
1723055252.6578014 INFO: reactions 1066
1723055252.7261033 INFO: Default biomass: [bio1]
1723055252.7763538 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055252.8001833 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055252.8051422 INFO: reactions 1300
1723055252.9736795 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1723055253.1904063 INFO: Default biomass: [bio1]
1723055253.2807436 INFO: Default biomass: [bio1]
1723055253.5302608 INFO: Default biomass: [bio1]
1723055253.587202 INFO: reactions 1021
1723055253.73062 INFO: The current solver interface glpk doesn't support setting the o

## combining the parallelized partitions

In [2]:
from json import load, dump
from glob import glob

gf_phenotype_results = {}
for path in glob(r"datacache/cleaned_SMIPPs/gf_phenotype_results_*.json"):
    with open(path, 'r') as jsonIn:
        gf_phenotype_results.update(load(jsonIn))
        
with open("datacache/cleaned_SMIPPs/gf_phenotype_results.json", 'w') as jsonOut:
    dump(gf_phenotype_results, jsonOut, indent=3)

# Computing MMIPPs

In [3]:
%run cliffcommutil.py
consolidated_gf_results = util.load("cleaned_SMIPPs/gf_phenotype_results")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
mag_abundances = util.load("mag_abundances")
mag_to_name = util.load("mag_to_name")
# total_mag_abundances = util.load("total_mag_abundances")
records = {
    "uptake_prob":[{"Sample":"Name"}],
    "excretion_prob":[{"Sample":"Name"}],
    "growth_prob":[{"Sample":"Name"}]
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for record in records:
        records[record][0][met] = metabolite_names[met]
for sample in mag_abundances:
    new_record = {
        "uptake_prob": {"Sample":sample},
        "excretion_prob": {"Sample":sample},
        "growth_prob": {"Sample":sample}
    }
    for metabolite in metabolites:
        for record in new_record:
            new_record[record][metabolite] = 0
    total_abundance = sum(list(mag_abundances[sample].values()))
    for OGname in mag_abundances[sample]:
        if OGname == "unbinned_reads":  continue
        # print(OGname)
        name = mag_to_name[OGname+".contigs__.RAST"]+".pg.G.D"
        # print(name)
        for metabolite in metabolites:
            for currtype in types:
                if currtype in consolidated_gf_results[name]:
                    if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                        new_record[currtype+"_prob"][metabolite] += mag_abundances[sample][OGname]/total_abundance*consolidated_gf_results[name][currtype][metabolite]["new_probability"]
    for record in records:
        records[record].append(new_record[record])
#Printing interation matrices
for record in records:
    df = pd.DataFrame.from_records(records[record])
    df.to_csv(util.output_dir+"/Sample"+record+"Interactions.csv",index=False)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Processing the data

In [7]:
%run cliffcommutil.py
consolidated_gf_results = util.load("cleaned_SMIPPs/gf_phenotype_results")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
mag_list = util.load("mag_list")
mag_to_name = util.load("mag_to_name")
records = {
    "uptake_prob":[{"MAG":"Name"}],
    "excretion_prob":[{"MAG":"Name"}],
    "growth_prob":[{"MAG":"Name"}],
    "uptake_data":[{"MAG":"Name"}],
    "excretion_data":[{"MAG":"Name"}],
    "growth_data":[{"MAG":"Name"}],
}
types = ["uptake","excretion","growth"]
for met in metabolites:
    for record in records:
        records[record][0][met] = metabolite_names[met]
for mag in mag_list:
    name = mag_to_name[mag[1]]+".pg.G.D"
    if name not in consolidated_gf_results:
        print("No data for ",name)
        continue
    new_record = {}
    for currtype in types:
        new_record[currtype+"_prob"] = {"MAG":name}
        new_record[currtype+"_data"] = {"MAG":name}
    for metabolite in metabolites:
        for currtype in types:
            if currtype in consolidated_gf_results[name]:
                if metabolite in consolidated_gf_results[name][currtype] and "new_probability" in consolidated_gf_results[name][currtype][metabolite]:
                    new_record[currtype+"_prob"][metabolite] = consolidated_gf_results[name][currtype][metabolite]["new_probability"]
                    new_record[currtype+"_data"][metabolite] = str(consolidated_gf_results[name][currtype][metabolite]["probrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["otherrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["origgfrxn"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["new_gf"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["gapfill_count"])
                    new_record[currtype+"_data"][metabolite] += "/"+str(consolidated_gf_results[name][currtype][metabolite]["reaction_count"])
    for record in records:
        records[record].append(new_record[record])
#Printing interation matrices
for record in records:
    df = pd.DataFrame.from_records(records[record])
    df.to_csv(util.output_dir+"/MAG"+record+"Interactions.csv",index=False)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


# Gapfilling models based on gapfilling simulation results

In [ ]:
%run cliffcommutil.py

def gapfillModels(arguments):
    (contents, name_to_mag, finished_models, metabolites, reactions_to_add, index) = arguments

    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    for OGname, fluxes in contents:
        # print(content[0], content[1].key())
        # OGname, fluxes = content
        name = name_to_mag[OGname.replace(".pg.G.D", '')]
        if name not in finished_models:
            mdlutl = util.msrecon.get_model(name.replace(".contigs__.RAST",".pangenome.RAST.glm4ec.pyr.O2"),186678)
            types = ["uptake","excretion","growth"]
            reactions_to_add[name] = {}
            for currtype in types:
                problist = []
                for metabolite in metabolites:
                    if metabolite in consolidated_gf_results[OGname][currtype] and "new_probability" in consolidated_gf_results[OGname][currtype][metabolite]:
                        if consolidated_gf_results[OGname][currtype][metabolite]["new_probability"] > 0:
                            problist.append(consolidated_gf_results[OGname][currtype][metabolite]["new_probability"])
                if len(problist) > 0:
                    ave = array(problist).mean()
                    stdev = array(problist).std()
                    for metabolite in metabolites:
                        if metabolite in consolidated_gf_results[OGname][currtype] and "new_probability" in consolidated_gf_results[OGname][currtype][metabolite]:
                            if consolidated_gf_results[OGname][currtype][metabolite]["new_probability"] > 0:
                                consolidated_gf_results[OGname][currtype][metabolite]["zscore"] = (consolidated_gf_results[OGname][currtype][metabolite]["new_probability"]-ave)/stdev
                                if consolidated_gf_results[OGname][currtype][metabolite]["zscore"] >= -1 and "reactions" in consolidated_gf_results[OGname][currtype][metabolite]:
                                    for rxn in consolidated_gf_results[OGname][currtype][metabolite]["reactions"]:
                                        direction = rxn[0:1]
                                        rxnid = rxn[1:]
                                        if rxnid not in mdlutl.model.reactions:
                                            if rxnid not in reactions_to_add[name]:
                                                reactions_to_add[name][rxnid] = {}
                                            reactions_to_add[name][rxnid][direction] = consolidated_gf_results[OGname][currtype][metabolite]["zscore"]
                                        elif direction == ">":
                                            if mdlutl.model.reactions.get_by_id(rxnid).upper_bound <= 0:
                                                if rxnid not in reactions_to_add[name]:
                                                    reactions_to_add[name][rxnid] = {}
                                                reactions_to_add[name][rxnid][direction] = consolidated_gf_results[OGname][currtype][metabolite]["zscore"]
                                        elif direction == "<":
                                            if mdlutl.model.reactions.get_by_id(rxnid).lower_bound >= 0:
                                                if rxnid not in reactions_to_add[name]:
                                                    reactions_to_add[name][rxnid] = {}
                                                reactions_to_add[name][rxnid][direction] = consolidated_gf_results[OGname][currtype][metabolite]["zscore"]
            util.save("cleaned_SMIPPs/gf_phenotype_results",consolidated_gf_results)
            msgapfill = MSGapfill(
                mdlutl,
                [util.msrecon.get_template(mdlutl.model.template_ref)],
                [],
                [],
                blacklist=[],
                default_target="bio1",
                minimum_obj=0.01,
                base_media=None,
                base_media_target_element=None
            )
            for cpd in metabolites:
                if "EX_"+cpd+"_e0" not in msgapfill.gfmodelutl.model.reactions:
                    transport = msgapfill.gfmodelutl.add_transport_and_exchange_for_metabolite(cpd,direction="=",prefix="trans",override=False)
            for rxnid in reactions_to_add[name]:
                if rxnid in msgapfill.gfmodel.reactions:
                    rxn = msgapfill.gfmodel.reactions.get_by_id(rxnid)
                    rxn = rxn.copy()
                    mdlutl.model.add_reactions([rxn])
                    rxn.upper_bound = 0
                    rxn.lower_bound = 0
                    if ">" in reactions_to_add[name][rxnid]:
                        rxn.upper_bound = 100
                    if "<" in reactions_to_add[name][rxnid]:
                        rxn.lower_bound = -100
                else:
                    print(name,"Missing reaction",rxnid,reactions_to_add[name][rxnid])
            util.msrecon.save_model(mdlutl,186678,name.replace(".contigs__.RAST",".pangenome.RAST.glm4ec.pyr.O2.gf"))
            finished_models[name] = 1
        util.save(f"finished_models_{index}",finished_models)
        util.save(f"reactions_to_add_{index}",reactions_to_add)


# define the parameters
consolidated_gf_results = util.load("cleaned_SMIPPs/gf_phenotype_results")
name_to_mag = util.load("name_to_mag")
finished_models = util.load("finished_models",{})
metabolites = util.load("metabolites")
reactions_to_add = util.load("reactions_to_add", {})

# run the parallelized code
parallelize = True
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split, array
    mag_results_sets = array_split(list(consolidated_gf_results.items()), int(len(consolidated_gf_results)/4))
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    pool_count = min(cpu_count(), len(mag_results_sets))
    print(f"{pool_count} threads will be created")
    pool = Pool(pool_count)
    args = [[content, name_to_mag, finished_models, metabolites, reactions_to_add, index] for index, content in enumerate(mag_results_sets)]
    list_of_outputs = pool.map(gapfillModels, args)
else:
    list_of_outputs = buildCommModel(list(mag_abundances.items()), mag_results_sets)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
77 threads will be created
ForkPoolWorker-78


1723601464.8968296 INFO: metabolites 1117


ForkPoolWorker-79
ForkPoolWorker-80


1723601465.3394754 INFO: metabolites 907


ForkPoolWorker-81


...

1723602983.8832846 WARNING: Ignoring reaction 'rxn00551_c0' since it already exists.
1723602984.5058677 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1723602990.3318837 INFO: cpd05178 not found in model!
1723602990.3337936 CRITICAL: Metabolite cpd05178 not found in model
1723602991.3271735 INFO: cpd05178 not found in model!
1723602991.329178 CRITICAL: Metabolite cpd05178 not found in model
1723602991.6368425 WARNING: Ignoring reaction 'rxn12215_c0' since it already exists.
1723602991.672429 WARNING: Ignoring reaction 'rxn04464_c0' since it already exists.
1723602991.7654483 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1723602992.3959608 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1723602993.1181664 INFO: cpd05178 not found in model!
1723602993.1201727 CRITICAL: Metabolite cpd05178 not found in model
1723602993.502509 WARNING: Ignoring reaction 'rxn00931_c0' since it already exists.


Salt_Pond_MetaG_R2_restored_C_black_MG_DASTool_bins_concoct_out.12.contigs__.RAST Missing reaction EX_cpd00557_c0 {'>': 1.1293953917264785}


1723602993.7184026 WARNING: Ignoring reaction 'rxn02285_c0' since it already exists.
1723602994.0001585 WARNING: Ignoring reaction 'rxn00929_c0' since it already exists.
1723602995.4736466 WARNING: Ignoring reaction 'rxn00551_c0' since it already exists.
1723602995.5448582 INFO: Setting FBAModel computed_attributes to mdlutl attributes
1723603010.6784449 INFO: cpd05178 not found in model!
1723603010.6804316 CRITICAL: Metabolite cpd05178 not found in model
1723603011.2326171 WARNING: Ignoring reaction 'rxn01048_c0' since it already exists.
1723603011.326532 WARNING: Ignoring reaction 'rxn00427_c0' since it already exists.


Salt_Pond_MetaGSF2_C_D2_MG_DASTool_bins_concoct_out.8.contigs__.RAST Missing reaction EX_cpd00557_c0 {'>': 1.0205293821453087}


1723603012.8832467 INFO: Setting FBAModel computed_attributes to mdlutl attributes


### combining the multiple gapfilling iterations

In [7]:
from glob import glob
from os import remove
from json import dump, load

print(len(glob("datacache/finished_models_*.json")))

finished_models_tot = {}
for finished_models in glob("datacache/finished_models_*.json"):
    with open(finished_models, 'r') as jsonIn:
        finished_models = load(jsonIn)
    finished_models_tot.update(finished_models)
    
with open("datacache/finished_models.json", 'w') as jsonOut:
    dump(finished_models_tot, jsonOut, indent=3)
    

print(len(glob("datacache/reactions_to_add_*.json")))

reactions_to_add_tot = {}
for reactions_to_add in glob("datacache/reactions_to_add_*.json"):
    with open(reactions_to_add, 'r') as jsonIn:
        reactions_to_add = load(jsonIn)
    reactions_to_add_tot.update(reactions_to_add)
    
with open("datacache/reactions_to_add.json", 'w') as jsonOut:
    dump(reactions_to_add_tot, jsonOut, indent=3)

77
77


## check that the gapfilling worked

In [5]:
%run cliffcommutil.py

dir(util.msrecon)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir


['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [18]:
%run cliffcommutil.py
from numpy import std, mean

# acquire the list of gapfilled content
salternModels = util.msrecon.kbase_api.list_objects(186678, object_type="KBaseFBA.FBAModel", include_metadata=True)
modelRxnCounts = {}
for model in salternModels:
    if "O2" not in model[1]:  continue
    if "gf" in model[1]:
        modelID = model[1].replace(".pangenome.RAST.glm4ec.pyr.O2.gf", '')
        if modelID not in modelRxnCounts:
            modelRxnCounts[modelID] = {"gapfilled": model[-1]['Number reactions']}
        else:
            modelRxnCounts[modelID].update({"gapfilled": model[-1]['Number reactions']})
    else:
        modelID = model[1].replace(".pangenome.RAST.glm4ec.pyr.O2", '')
        if modelID not in modelRxnCounts:
            modelRxnCounts[modelID] = {"base": model[-1]['Number reactions']}
        else:
            modelRxnCounts[modelID].update({"base": model[-1]['Number reactions']})
    
# determine the gapfilling statistics 
totRxns, reactionStats = {"gapfilled": [], "base": []}, {"gapfilled": {}, "base": {}}
badModels = []
for model, content in modelRxnCounts.items():
    totRxns["base"].append(content["base"])
    if "gapfilled" not in content:
        print(f"{model} does not have a gapfilled model")
        badModels.append(model)
        continue
    totRxns["gapfilled"].append(content["gapfilled"])

for category, rxnNums in totRxns.items():
    vals = [int(x) for x in rxnNums]
    reactionStats[category] = {
        "num": len(rxnNums),
        "ave": mean(vals),
        "std": std(vals)
    }
# print(totRxns)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
Salt_Pond_MetaG_R2_C_H2O_MG_DASTool_bins_metabat.20 does not have a gapfilled model
Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.9 does not have a gapfilled model
Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_concoct_out.53 does not have a gapfilled model
Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.11 does not have a gapfilled model


In [19]:
display(badModels, reactionStats)

['Salt_Pond_MetaG_R2_C_H2O_MG_DASTool_bins_metabat.20',
 'Salt_Pond_MetaG_R2_A_D1_MG_DASTool_bins_metabat.9',
 'Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_concoct_out.53',
 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.11']

{'gapfilled': {'num': 304,
  'ave': 1226.3355263157894,
  'std': 153.49742600614664},
 'base': {'num': 308, 'ave': 1009.2175324675325, 'std': 157.55117108671203}}

# Building and saving community model

### abundaces adjustment

In [3]:
from json import load, dump

abundancesDic = load(open("mag_abundances.json"))
refinedAbundances = {}
for sample, abundances in abundancesDic.items():
    refinedAbundances[sample] = {}
    abundances.pop("unbinned_reads")
    totAbundance = sum(list(abundances.values()))
    for mag, abundance in abundances.items():
        refinedAbundances[sample][mag] = abundance/totAbundance
    print(sum(list(refinedAbundances[sample].values())))

dump(refinedAbundances, open("datacache/refined_MAG_abundances.json", 'w'), indent=3)

0.9999999999999994
1.0000000000000002
1.000000000000001
0.9999999999999969
0.9999999999999988
0.9999999999999993
0.9999999999999999
1.0
1.0000000000000002
0.9999999999999999
1.0000000000000004
1.0000000000000009
1.0000000000000009
1.0000000000000007
0.9999999999999996
1.0000000000000004
1.0
0.9999999999999996
1.0
1.0
1.0000000000000007
0.9999999999999998
0.9999999999999998
1.0000000000000007


## create the models

### APF version

In [1]:
%run cliffcommutil.py



def buildCommModel(arguments):
    (mag_abundances, mag_list) = arguments
    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    # build the community model from the members
    for info in mag_abundances:
        sample, abundances = info
        member_models = []
        member_names = []
        member_abundances = {}
        ## create the member abundances dictionary
        for mag, abundance in abundances.items():
            if abundance < 0.01:  continue
            mdlutl = util.msrecon.get_model(mag+".pangenome.RAST.glm4ec.pyr.O2.gf",186678)
            member_models.append(mdlutl.model)
            member_names.append(mag)
            member_abundances[mag] = abundance
        comm_model = build_from_species_models(
            member_models,
            model_id=sample,
            name=sample,
            abundances=member_abundances
        )
        cobra.io.save_json_model(comm_model, "models/"+sample+'_APF.json')
    

import cobra
mag_list = util.load("mag_list")
mag_abundances = util.load("refined_MAG_abundances")

# run the parallelized code
parallelize = False
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    pool_count = min(cpu_count(), len(mag_abundances))
    print(f"{pool_count} threads will be created")
    pool = Pool(pool_count)
    args = [[[abundances], mag_list] for abundances in list(mag_abundances.items())]
    list_of_outputs = pool.map(buildCommModel, args)
else:
    list_of_outputs = buildCommModel([list(mag_abundances.items()), mag_list])

python version 3.11.1
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.4.0
Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
MainProcess


1724795223.8013704 INFO: metabolites 1261
1724795224.3416324 INFO: reactions 1292
1724795224.4795008 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724795225.3879337 INFO: Default biomass: [bio1]
1724795227.0119321 INFO: metabolites 1196
1724795227.613509 INFO: reactions 1264
1724795227.761144 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724795228.6444411 INFO: Default biomass: [bio1]
1724795229.1647696 INFO: metabolites 1285
1724795229.8651652 INFO: reactions 1315
1724795230.0185068 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724795230.8617983 INFO: Default biomass: [bio1]
1724795231.3667727 INFO: metabolites 1243
1724795231.971922 INFO: reactions 1303
1724795232.1094666 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724795232.6112962 INFO: Default biomass: [bio1]
1724795233.6646695 INFO: metabolites 1140
1

TypeError: MSModelUtil.__init__() got an unexpected keyword argument 'copy'

### CSH version

In [1]:
%run cliffcommutil.py

def buildCommModel(arguments):
    (mag_abundances, mag_list) = arguments
    
    # print status
    from multiprocess import current_process
    pid = current_process().name
    print(pid)
    
    # build the community model from the members
    for info in mag_abundances:
        sample, abundances = info
        member_models = []
        member_names = []
        member_abundances = {}
        for mag, abundance in abundances.items():
            if abundance < 0.01:  continue
            try:
                mdlutl = util.msrecon.get_model(mag+".pangenome.RAST.glm4ec.pyr.O2.gf",186678)
            except:
                print(f"The {mag}.pangenome.RAST.glm4ec.pyr.O2.gf is not in 186678")
                continue
            member_models.append(mdlutl.model)
            member_names.append(mag)
            member_abundances[mag] = abundance
        comm_model = MSCommunity.build_from_species_models(
            member_models,
            mdlid=sample,
            name=sample,
            names=member_names,
            abundances=member_abundances
        )
        cobra.io.save_json_model(comm_model.model, "models/"+sample+'.json')
    

import cobra
failingModels = ["SF2_A_D2", "SF2_C_D1", "SF2_B_D2", "R2_A_D1", "R2_B_D1", "SF2_C_D2",
                 "R2_C_D1", "R2_C_D2", "R2_A_D2", "R2_B_D2", "R1_C_D1", "R1_A_D1"]
mag_list = util.load("mag_list")
mag_abundances = util.load("refined_MAG_abundances")

# run the parallelized code
parallelize = True
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    
    # the MAGs are partitioned into groups of 4, and a thread is created to handle each group 
    pool_count = min(cpu_count(), len(mag_abundances))
    print(f"{pool_count} threads will be created")
    pool = Pool(pool_count)
    args = [[[abundances], mag_list] for abundances in list(mag_abundances.items())]
    list_of_outputs = pool.map(buildCommModel, args)
else:
    list_of_outputs = buildCommModel([list(mag_abundances.items()), mag_list])

python version 3.11.1
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.4.0
Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
24 threads will be created
ForkPoolWorker-1


ForkPoolWorker-2ForkPoolWorker-3
ForkPoolWorker-4ForkPoolWorker-5
ForkPoolWorker-6
ForkPoolWorker-7
ForkPoolWorker-8
ForkPoolWorker-9
ForkPoolWorker-10
ForkPoolWorker-11
ForkPoolWorker-12
ForkPoolWorker-13
ForkPoolWorker-14
ForkPoolWorker-15
ForkPoolWorker-16
ForkPoolWorker-17
ForkPoolWorker-18
ForkPoolWorker-19
ForkPoolWorker-20
ForkPoolWorker-21
ForkPoolWorker-22
ForkPoolWorker-23
ForkPoolWorker-24
The Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_concoct_out.53.pangenome.RAST.glm4ec.pyr.O2.gf is not in 186678
The Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_concoct_out.53.pangenome.RAST.glm4ec.pyr.O2.gf is not in 186678


1724811750.6927388 INFO: metabolites 1261
1724811750.828401 INFO: metabolites 1170
1724811750.8623137 INFO: metabolites 1261
1724811750.8788252 INFO: metabolites 1170
1724811750.9430435 INFO: metabolites 1170
1724811750.9522645 INFO: metabolites 1170
1724811750.9822638 INFO: metabolites 1261
1724811750.9978032 INFO: metabolites 1170
1724811751.0060813 INFO: metabolites 1152
1724811751.0129082 INFO: metabolites 1152
1724811751.1252513 INFO: metabolites 1305
1724811751.1525526 INFO: metabolites 1140
1724811751.1589227 INFO: metabolites 1261
1724811751.1681464 INFO: metabolites 1261
1724811751.182422 INFO: metabolites 1327
1724811751.1877236 INFO: metabolites 1261
1724811751.193667 INFO: metabolites 1152
1724811751.1979263 INFO: metabolites 1266
1724811751.2181718 INFO: metabolites 1177
1724811751.2221508 INFO: metabolites 1024
1724811751.235486 INFO: metabolites 1318
1724811751.2357593 INFO: reactions 1292
1724811751.3301678 INFO: metabolites 1261
1724811751.3726609 INFO: The current sol

The Salt_Pond_MetaG_R1_B_D1_MG_DASTool_bins_concoct_out.53.pangenome.RAST.glm4ec.pyr.O2.gf is not in 186678


1724811752.5107121 INFO: Default biomass: [bio1]
1724811752.5191655 INFO: Default biomass: [bio1]
1724811752.5218232 INFO: Default biomass: [bio1]
1724811752.5494845 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724811752.6001022 INFO: Default biomass: [bio1]
1724811752.6484835 INFO: Default biomass: [bio1]
1724811752.686598 INFO: Default biomass: [bio1]
1724811752.6913562 INFO: Default biomass: [bio1]
1724811752.7328773 INFO: metabolites 1196
1724811752.7489865 INFO: Default biomass: [bio1]
1724811752.756853 INFO: Default biomass: [bio1]
1724811752.7751606 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1724811752.7763119 INFO: Default biomass: [bio1]
1724811752.8619378 INFO: Default biomass: [bio1]
1724811752.9340322 INFO: metabolites 1134
1724811752.9383156 INFO: Default biomass: [bio1]
1724811752.9387934 INFO: Default biomass: [bio1]


The Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.11.pangenome.RAST.glm4ec.pyr.O2.gf is not in 186678


...

32 Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.45
30 Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.100


1724811948.4044144 INFO: Making atp hydrolysis reaction for species: Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.45


31 Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.45


1724811949.7143521 INFO: Making atp hydrolysis reaction for species: Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.47


32 Salt_Pond_MetaG_R2_C_D2_MG_DASTool_bins_concoct_out.47


## compute community model data

In [ ]:
%run agoracommutil.py
import cobra
sample_abundanes = util.load("mag_abundanes")

community_model_data = {}
for pair in sample_pairs:
    member_models = []
    member_names = []
    member_abundances = {}
    asv_hash = {}
    for asvset in sample_asvset_abundances[pair[0]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[0]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[0]][asvset]
    for asvset in sample_asvset_abundances[pair[1]]:
        asvname = asvset_names[asvset]
        if asvname not in asv_hash:
            asv_hash[asvname] = sample_asvset_abundances[pair[1]][asvset]
        else:
            asv_hash[asvname] += sample_asvset_abundances[pair[1]][asvset]
    total_abundance = 0
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/2
        total_abundance += asv_hash[asvname]
    for asvname in asv_hash:
        asv_hash[asvname] = asv_hash[asvname]/total_abundance
        if asvname != "Thermus.1" and asv_hash[asvname] > 0.01:
            member_names.append(asvname)
            member_abundances[asvname] = asv_hash[asvname]
    community_model_data[pair[2]] = {
        "names":member_names,
        "abundances":member_abundances
    }
    util.save("community_model_data",community_model_data)
    

## community model statistics

In [ ]:
from json import load
from glob import glob

modelStats = {}
for path in glob("models/*.json"):
    modelStats[path] = {}
    for rxn in modelStats["reactions"]:
        ID, compartment = rxn.split("_")
        modelStats[path][compartment] += 1
    
modelStats

# Running community model

In [26]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {"NotebookApp": {"iopub_data_rate_limit": 10000000}})

{'NotebookApp': {'iopub_data_rate_limit': 10000000}}

## determine the top hits for each methane flux regime

In [1]:
from modelseedpy.biochem import from_local
msdb = from_local("/scratch/shared/code/ModelSEEDDatabase")
# print(msdb.compounds.keys())

modelseedpy 0.4.0


In [17]:
sampleRegime = {'R2A_A_D1': 'red',
 'R2A_A_D2': 'red',
 'R2A_B_D1': 'red',
 'R2A_B_D2': 'red',
 'R2A_C_D1': 'red',
 'R2A_C_D2': 'red',
 'SF2_A_D1': 'red',
 'SF2_A_D2': 'red',
 'SF2_B_D1': 'red',
 'SF2_B_D2': 'red',
 'SF2_C_D1': 'red',
 'SF2_C_D2': 'red',
 'R1_A_D1': 'green',
 'R1_A_D2': 'green',
 'R1_B_D1': 'blue',
 'R1_B_D2': 'blue',
 'R1_C_D1': 'green',
 'R1_C_D2': 'green',
 'R2_A_D1': 'blue',
 'R2_A_D2': 'blue',
 'R2_B_D1': 'purple',
 'R2_B_D2': 'purple',
 'R2_C_D1': 'purple',
 'R2_C_D2': 'purple'}

# load and process the MMIPPs correlations
from pandas import read_csv
redDF = read_csv("nboutput/CH4_MMIPPs_correlations_red.csv").set_index("Unnamed: 0").rename_axis('Metabolites')
greenDF = read_csv("nboutput/CH4_MMIPPs_correlations_green.csv").set_index("Unnamed: 0").rename_axis('Metabolites')
blueDF = read_csv("nboutput/CH4_MMIPPs_correlations_blue.csv").set_index("Unnamed: 0").rename_axis('Metabolites')
purpleDF = read_csv("nboutput/CH4_MMIPPs_correlations_purple.csv").set_index("Unnamed: 0").rename_axis('Metabolites')

# redDF["totalEX"] = redDF["Sampleexcretion_probInteractions"] + redDF["Sampleuptake_probInteractions"]
# greenDF["totalEX"] = greenDF["Sampleexcretion_probInteractions"] + greenDF["Sampleuptake_probInteractions"]
# blueDF["totalEX"] = blueDF["Sampleexcretion_probInteractions"] + blueDF["Sampleuptake_probInteractions"]
# purpleDF["totalEX"] = purpleDF["Sampleexcretion_probInteractions"] + purpleDF["Sampleuptake_probInteractions"]

colToSort = "Sampleuptake_probInteractions"
redDF = redDF.sort_values(by=colToSort, ascending=False)
greenDF = greenDF.sort_values(by=colToSort, ascending=False)
blueDF = blueDF.sort_values(by=colToSort, ascending=False)
purpleDF = purpleDF.sort_values(by=colToSort, ascending=False)

# parse and convert the top 10 correlating compounds by uptake and excretion
# load the compound categories
from json import load
cpdNames = load(open("/scratch/shared/code/CommScores/data/compoundNames.json", 'r'))
compounds = load(open("data/metabolomics_categories.json", 'r'))
metCategory = {v:k for k,vals in compounds.items() for v in vals}
new_compounds = {}
for category, mets in compounds.items():
    new_compounds[category] = {cpdNames[met]:met for met in mets}
# print(new_compounds["Amino Acids"])
    
from numpy import load
allCpds = {"Amino Acids": dict(load("/scratch/shared/code/CommScores/data/categories/aminoAcids.npy")),
          # "Energy": load("/scratch/shared/code/CommScores/data/categories/energy_compounds.npy"),
          "Minerals": dict(load("/scratch/shared/code/CommScores/data/categories/minerals.npy")),
          "Sugars": dict(load("/scratch/shared/code/CommScores/data/categories/sugars.npy")),
          "Vitamins": dict(load("/scratch/shared/code/CommScores/data/categories/vitamins.npy"))}
allCpds.update(new_compounds)
allCpdsCategory = {v:k for k,vals in allCpds.items() for v in vals}

# print(allCpds["Amino Acids"])
redTop, greenTop, blueTop, purpleTop = [], [], [], []
tops = [redTop, greenTop, blueTop, purpleTop]
missingCpds = []
for i, index in enumerate([redDF.index, greenDF.index, blueDF.index, purpleDF.index]):
    for cpd in index:
        if cpd not in allCpdsCategory:
            missingCpds.append(cpd)  ;  continue
        category = allCpdsCategory[cpd]
        if category in ["Amino Acids"]: continue
        tops[i].append(allCpds[category][cpd])
        if len(tops[i]) > 10: break
    
# redTop = [msdb.compounds[cpd]["name"] for cpd in list(redDF.index[:10])]
# greenTop = [msdb.compounds[cpd]["name"] for cpd in list(greenDF.index[:10])]
# blueTop = [msdb.compounds[cpd]["name"] for cpd in list(blueDF.index[:10])]
# purpleTop = [msdb.compounds[cpd]["name"] for cpd in list(purpleDF.index[:10])]

from icecream import ic
ic(redTop, greenTop, blueTop, purpleTop)

ic| redTop: ['L-Fructose',
             'D-Glucose',
             'Propionate',
             '2-Oxoglutarate',
             'Fumarate',
             'Sucrose',
             'Maltose',
             'Acetate',
             'Succinate']
    greenTop: ['L-Fructose',
               'Succinate',
               'Maltose',
               'Acetate',
               'D-Glucose',
               'Sucrose',
               '2-Oxoglutarate',
               'Propionate',
               'Fumarate']
    blueTop: ['D-Glucose',
              'Succinate',
              'Maltose',
              'Acetate',
              'Fumarate',
              'Propionate',
              'Sucrose',
              '2-Oxoglutarate',
              'L-Fructose']
    purpleTop: ['2-Oxoglutarate',
                'Succinate',
                'Fumarate',
                'L-Fructose',
                'Sucrose',
                'Acetate',
                'Propionate',
                'D-Glucose',
                'Maltose']


(['L-Fructose',
  'D-Glucose',
  'Propionate',
  '2-Oxoglutarate',
  'Fumarate',
  'Sucrose',
  'Maltose',
  'Acetate',
  'Succinate'],
 ['L-Fructose',
  'Succinate',
  'Maltose',
  'Acetate',
  'D-Glucose',
  'Sucrose',
  '2-Oxoglutarate',
  'Propionate',
  'Fumarate'],
 ['D-Glucose',
  'Succinate',
  'Maltose',
  'Acetate',
  'Fumarate',
  'Propionate',
  'Sucrose',
  '2-Oxoglutarate',
  'L-Fructose'],
 ['2-Oxoglutarate',
  'Succinate',
  'Fumarate',
  'L-Fructose',
  'Sucrose',
  'Acetate',
  'Propionate',
  'D-Glucose',
  'Maltose'])

In [20]:
unionCPDs = set(redTop + greenTop + blueTop + purpleTop)
print(len(unionCPDs), unionCPDs)

9 {'Sucrose', 'Succinate', 'Maltose', 'Fumarate', 'L-Fructose', 'Acetate', 'Propionate', 'D-Glucose', '2-Oxoglutarate'}


## run the model

In [ ]:
# %run cliffcommutil.py

# def runCommModel(arguments):
#     from multiprocess import current_process
#     from optlang.symbolics import Zero, add
    
#     print(arguments)
#     (model_paths, feature_probabilities, metabolomics_data, metabolites, dipeptide_exchanges, media) = arguments
#     output, rxn_record_hash = {}, {}
#     modelIDs = []
#     records = util.load("records")
#     for model_path in model_paths:
#         # print status
#         pid = current_process().name
#         print(pid)

#         modelID = model_path.split("/")[-1].replace(".json", '')
#         modelIDs.append(modelID)
#         output[modelID] = {}
#         base_model = load_json_model(model_path)
#         current_comm_model = MSCommunity(
#             model=base_model,
#         )
#         # print(dir(current_comm_model))
#         # assign reaction probabilities
#         reaction_probabilities = {}
#         for rxn in base_model.reactions:
#             highest_prob = None
#             for gene in rxn.genes:
#                 if gene.id in feature_probabilities:
#                     if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
#                         highest_prob = feature_probabilities[gene.id]
#             if highest_prob != None:
#                 rxn.probability = highest_prob
#                 reaction_probabilities[rxn.id] = highest_prob
#             elif rxn.id[0:3] != "bio" and rxn.id[0:3] != "EX_" and rxn.id[0:3] != "DM_" and len(rxn.genes) == 0:
#                 reaction_probabilities[rxn.id] = 0
#             else:
#                 reaction_probabilities[rxn.id] = 0.05
#         min_prob, prob_exp, ex_weight = 0.05, 1, 1
#         mdlutl = current_comm_model.mdlutl
#         pkgmgr = MSPackageManager.get_pkg_mgr(mdlutl)
#         # add media, community kinetics, and elemental uptake constraints
#         pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
#         pkgmgr.getpkg("CommKineticPkg").build_package(750, current_comm_model)
#         cUptake = 500
#         pkgmgr.getpkg("ElementUptakePkg").build_package({"C": cUptake})
#         # update exchange flux bounds 
#         for rxn in base_model.reactions:
#             if "EX_" != rxn.id[0:3]:   continue
#             currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
#             if rxn.id in dipeptide_exchanges:
#                 currRxn.lower_bound = 0
#                 currRxn.upper_bound = 0
#             else:
#                 ## the bounds are set to either -1000 or 1000
#                 if currRxn.lower_bound < 0:
#                     currRxn.lower_bound = -1000
#                 if currRxn.upper_bound > 0:
#                     currRxn.upper_bound = 1000
#         if "EX_cpd00007_e0" in [rxn.id for rxn in mdlutl.exchange_list()]:
#             mdlutl.model.reactions.get_by_id("EX_cpd00007_e0").lower_bound = -cUptake/3   # oxygen
#         else:
#             print(f"The {mdlutl.model.id} does not have Oxygen defined in the model")

#         # maximize biomass production and store the results
#         mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="max")
#         mdlutl.model.objective.set_linear_coefficients({mdlutl.model.reactions.bio1.forward_variable: 1})
#         maxBiomassSol = mdlutl.model.optimize()
#         output[modelID]["max_growth"] = maxBiomassSol.objective_value
#         output[modelID]["carbon_uptake"] = pkgmgr.getpkg("ElementUptakePkg").variables["elements"]["C"].primal
#         records[0][modelID] = output[modelID]["max_growth"]
#         records[1][modelID] = output[modelID]["carbon_uptake"]
#         print(modelID+" growth:", output[modelID]["max_growth"])
#         print(modelID+"carbon uptake:",output[modelID]["carbon_uptake"])
#         ## export the LP
#         export_path = util.output_dir+"/LPs/"+modelID+"-growth.lp"
#         if not os.path.exists(export_path):
#             os.makedirs(export_path)
#         with open(export_path, "w") as out:
#             out.write(str(mdlutl.model.solver))
#         if str(output[modelID]["max_growth"]) == "nan":
#             print("Skipping condition due to infeasibility", modelID)
# [modelID]output[modelID]["carbon_uptake"])
#             return

#         # simulate the community with the specified community growth
#         for percentOptimum in [1, 0.9, 0.8, 0.7, 0.6, 0.5]:
#             mdlutl.add_minimal_objective_cons(output[modelID]["max_growth"] * percentOptimum)
#             # pkgmgr.getpkg("ObjConstPkg").build_package(, None)
#             ## defining the minimal probability objective
#             output[modelID][percentOptimum] = {}
#             coef, flux_fitting_target = {}, {}
#             for rxn in base_model.reactions:
#                 if "rxn" == rxn.id[0:3]:
#                     prob = reaction_probabilities[rxn.id]
#                     currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
#                     coef.update({currRxn.forward_variable: max(min_prob, (1 - float(prob) ** prob_exp))})
#                     coef.update({currRxn.reverse_variable: max(min_prob, (1 - float(prob) ** prob_exp))})
#                 elif "EX_" == rxn.id[0:3]:
#                     currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
#                     if rxn.id[3:11] in metabolomics_data:
#                         flux_fitting_target[rxn.id] = -1*output[modelID]["max_growth"]*metabolomics_data[rxn.id[3:11]][modelID]
#                     coef.update({currRxn.forward_variable: ex_weight})
#                     coef.update({currRxn.reverse_variable: ex_weight})

#             ## Adjusting rxn bounds according to the maximum community growth and metabolomics data
#             stuck_reactions = {}
#             for iteration in range(1,11,1):
#                 print("Iteration",iteration)
#                 for rxn in flux_fitting_target:
#                     if rxn in stuck_reactions:   continue
#                     currRxn = mdlutl.model.reactions.get_by_id(rxn)
#                     starting_point = maxBiomassSol.fluxes[rxn]
#                     distance = abs(flux_fitting_target[rxn] - starting_point)
#                     original_upper = currRxn.upper_bound
#                     original_lower = currRxn.lower_bound
#                     if starting_point > flux_fitting_target[rxn]:
#                         if starting_point-iteration*distance/10 < currRxn.upper_bound:
#                             currRxn.lower_bound = starting_point-iteration*distance/10
#                             currRxn.upper_bound = starting_point-iteration*distance/10
#                         else:
#                             currRxn.upper_bound = starting_point-iteration*distance/10
#                             currRxn.lower_bound = starting_point-iteration*distance/10
#                     else:
#                         if flux_fitting_target[rxn]-(10-iteration)*distance/10 < currRxn.upper_bound:
#                             currRxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
#                             currRxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
#                         else:
#                             currRxn.upper_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
#                             currRxn.lower_bound = flux_fitting_target[rxn]-(10-iteration)*distance/10
#                     solution = mdlutl.model.optimize()
#                     if solution.status != "optimal":
#                         print("Stuck",rxn,starting_point,flux_fitting_target[rxn],original_upper)
#                         stuck_reactions[rxn] = 1
#                         if original_upper >= currRxn.lower_bound:
#                             currRxn.upper_bound = original_upper
#                             currRxn.lower_bound = original_lower
#                         else:
#                             currRxn.lower_bound = original_lower
#                             currRxn.upper_bound = original_upper
#             for rxn in flux_fitting_target:
#                 currRxn = mdlutl.model.reactions.get_by_id(rxn)
#                 print(currRxn.id,currRxn.lower_bound,currRxn.upper_bound)

#             ## maximize methane excretion
#             ch4_weight = 100
#             if "EX_cpd01024_e0" not in [rxn.id for rxn in mdlutl.exchange_list()]:
#                 print(f"The {mdlutl.model.id} does not have Methane defined in the model")
#             else:
#                 methane = mdlutl.model.reactions.get_by_id("EX_cpd01024_e0")
#                 coef.update({methane.reverse_variable: ch4_weight, methane.forward_variable: -ch4_weight})

#             ## assigning the minimum objective function
#             mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="min")
#             mdlutl.model.objective.set_linear_coefficients(coef)

#             ## simulating and checking the solution
#             solution = mdlutl.model.optimize()
#             if solution.status != "optimal":
#                 continue
#                 # while solution.status != "optimal":
#                 #     util.save("objConstPkg_contents", mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"])
#                 #     print(dir(mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]),
#                 #           mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"])
#                 #     print("Infeasible: reducing objective constraint",0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb)
#                 #     mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb = 0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb
#                 #     solution = mdlutl.model.optimize()

#             ## storing and exporting successful simulation results
#             export_path = util.output_dir+"/LPs/"+modelID+"-final.lp"
#             if not os.path.exists(export_path):
#                 os.makedirs(export_path)
#             with open(export_path, "w") as out:
#                 out.write(str(mdlutl.model.solver))

#             output[modelID][percentOptimum]["minimum probability objective"] = solution.objective_value
#             records[3][modelID] = output[modelID]["minimum probability objective"]
#             output[modelID][percentOptimum]["solution"] = {}
#             for rxn in mdlutl.model.reactions:
#                 if rxn.id not in rxn_record_hash:
#                     rxn_record_hash[rxn.id] = {"id":rxn.id}
#                     for mdlID in modelIDs:
#                         rxn_record_hash[rxn.id][mdlID] = None
#                     records.append(rxn_record_hash[rxn.id])
#                 rxn_record_hash[rxn.id][modelID] = solution.fluxes[rxn.id]
#                 output[modelID][percentOptimum]["solution"][rxn.id] = solution.fluxes[rxn.id]
            
#             dirName = util.output_dir+f"/{modelID}"
#             if not os.path.exists(dirName):
#                 os.makedirs(dirName)
#             baseName = dirName+f"/{percentOptimum*100}%"
#             with open(baseName+".json", "w") as out:
#                 dump(output, out, indent=3)
#             df = DataFrame.from_records(records)
#             df.to_csv(f"{baseName}.csv")
        
#         with open(path.join(util.output_dir, "outputs", f"{modelID}_output.json"), "w") as out:
#             dump(output, out, indent=3)
#         with open(path.join(util.output_dir, "outputs", f"{modelID}_rxn_record_hash.json"), "w") as out:
#             dump(rxn_record_hash, out, indent=3)
#         util.save(f"{modelID}_records", records)

# # import stored data
# metabolites = util.load("metabolites")
# metabolomics_path = "/scratch/shared/code/MicrobiomeNotebooks/Cliff/data/Cliff_Sample_Metadata_BGC_NMR.csv"
# metabolomics_data = read_csv(metabolomics_path).drop(["New_index", "Unnamed: 0"], axis=1).set_index("Sample").to_dict()
# feature_probabilities = util.load("feature_probabilities")
# complete_media = util.msrecon.get_media("KBaseMedia/Complete")


# # R1_B_D2 is relaly slow for some reason

# from glob import glob
# from cobra.io import load_json_model
# from json import load, dump
# with open("/scratch/shared/code/MicrobiomeNotebooks/Cliff/data/dipeptide_exchanges.json", 'r') as jsonIn:
#     dipeptide_exchanges = load(jsonIn)
# parallelize = True
# if parallelize:
#     from multiprocess import Pool
#     from os import cpu_count
#     from numpy import array_split
    
#     # a thread is given for each community model
#     pool_count = min(cpu_count(), len(list(glob("models/*.json"))))
#     print(f"{pool_count} threads will be created")
#     pool = Pool(pool_count)
#     args = [[[model_path], feature_probabilities, metabolomics_data, metabolites, dipeptide_exchanges, complete_media]
#             for model_path in list(glob("models/*.json"))]
#     list_of_outputs = pool.map(runCommModel, args)
# else:
#     outputs = runCommModel([list(glob("models/*.json")), feature_probabilities,
#                             metabolomics_data, metabolites, dipeptide_exchanges, complete_media])

In [ ]:
%run cliffcommutil.py

def runCommModel(arguments):
    from multiprocess import current_process
    from optlang.symbolics import Zero, add
    
    print(arguments)
    (model_paths, feature_probabilities, metabolomics_data, metabolites, dipeptide_exchanges, media,
     commKin, cUptake, missing) = arguments
    output, rxn_record_hash = {}, {}
    modelIDs = []
    records = [{"id":"max_growth"}, {"id":"carbon_uptake"}, {"id":"flux fitting objective"}, {"id":"minimum probability objective"}]
    
    # simulate the community with the specified community growth
    percentOptimums = [1, 0.9, 0.8, 0.7, 0.6, 0.5]
    for percentOptimum in percentOptimums:
        optStr = str(int(percentOptimum*100))
        if missing is not None and optStr not in missing:  continue
        output[percentOptimum] = {}
        dirName = util.output_dir+f"/{int(percentOptimum*100)}"
        if not os.path.exists(dirName):
            os.makedirs(dirName)
        for model_path in model_paths:
            # print status
            pid = current_process().name
            print(pid, model_path)

            modelID = model_path.split("/")[-1].replace(".json", '')
            if missing is not None and modelID not in missing[optStr]:
                print(f"{percentOptimum} missing {modelID}")
                continue
            modelIDs.append(modelID)
            output[percentOptimum][modelID] = {}
            base_model = load_json_model(model_path)
            print(type(base_model.solver))
            current_comm_model = MSCommunity(model=base_model)
            # print(dir(current_comm_model))
            # assign reaction probabilities
            reaction_probabilities = {}
            for rxn in base_model.reactions:
                # highest_prob = max([feature_probabilities.get(gene.id) for gene in rxn.genes])
                highest_prob = None
                for gene in rxn.genes:
                    if gene.id in feature_probabilities:
                        if highest_prob == None or feature_probabilities[gene.id] > highest_prob:
                            highest_prob = feature_probabilities[gene.id]
                if highest_prob != None:
                    rxn.probability = highest_prob
                    reaction_probabilities[rxn.id] = highest_prob
                elif not any([rxn.id[0:3] in ["bio", "EX_", "DM_"]]) and len(rxn.genes) == 0:
                    reaction_probabilities[rxn.id] = 0
                else:
                    reaction_probabilities[rxn.id] = 0.05
            min_prob, prob_exp, ex_weight = 0.05, 1, 1
            mdlutl = current_comm_model.mdlutl
            pkgmgr = MSPackageManager.get_pkg_mgr(mdlutl)
            # add media, community kinetics, and elemental uptake constraints
            pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
            pkgmgr.getpkg("CommKineticPkg").build_package(commKin, current_comm_model)
            pkgmgr.getpkg("ElementUptakePkg").build_package({"C": cUptake})
            # update exchange flux bounds 
            for rxn in base_model.reactions:
                if "EX_" != rxn.id[0:3]:   continue
                currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
                if rxn.id in dipeptide_exchanges:
                    currRxn.lower_bound = 0
                    currRxn.upper_bound = 0
                else:
                    ## the bounds are set to either -1000 or 1000
                    if currRxn.lower_bound < 0:
                        currRxn.lower_bound = -1000
                    if currRxn.upper_bound > 0:
                        currRxn.upper_bound = 1000
            if "EX_cpd00007_e0" in [rxn.id for rxn in mdlutl.exchange_list()]:
                mdlutl.model.reactions.get_by_id("EX_cpd00007_e0").lower_bound = -cUptake/3   # oxygen
            else:
                print(f"The {mdlutl.model.id} does not have Oxygen defined in the model")

            # optimizations
            ## maximize biomass production and store the results
            mdlutl.add_objective(Zero, "max", {mdlutl.model.reactions.bio1.forward_variable: 1})
            maxBiomassSol = mdlutl.model.optimize()
            output[percentOptimum][modelID]["max_growth"] = maxBiomassSol.objective_value
            output[percentOptimum][modelID]["carbon_uptake"] = pkgmgr.getpkg("ElementUptakePkg").variables["elements"]["C"].primal
            records[0][modelID] = output[percentOptimum][modelID]["max_growth"]
            records[1][modelID] = output[percentOptimum][modelID]["carbon_uptake"]
            print(modelID+" growth:", output[percentOptimum][modelID]["max_growth"])
            print(modelID+"carbon uptake:", output[percentOptimum][modelID]["carbon_uptake"])
            ## export the LP
            export_path = util.output_dir+"/LPs/"+modelID+"-growth.lp"
            if not os.path.exists(export_path):
                os.makedirs(export_path)
            with open(export_path, "w") as out:
                out.write(str(mdlutl.model.solver))
            if str(output[percentOptimum][modelID]["max_growth"]) == "nan":
                print("Skipping condition due to infeasibility", modelID)
                return
            mdlutl.add_minimal_objective_cons(output[percentOptimum][modelID]["max_growth"] * percentOptimum, name="minGrowth")
            # pkgmgr.getpkg("ObjConstPkg").build_package(, None)
            
            ## maximize methane excretion
            mdlutl.add_objective(Zero, "max", {mdlutl.model.reactions.get_by_id("EX_cpd01024_e0").forward_variable: 1})
            maxMethaneSol = mdlutl.model.optimize()
            output[percentOptimum][modelID]["max_methane"] = maxMethaneSol.objective_value
            print(modelID, "methane flux:", output[percentOptimum][modelID]["max_methane"])
            mdlutl.add_minimal_objective_cons(output[percentOptimum][modelID]["max_methane"], name="minCH4")
            # ch4_weight = 100
            # if "EX_cpd01024_e0" not in [rxn.id for rxn in mdlutl.exchange_list()]:
            #     print(f"The {mdlutl.model.id} does not have Methane defined in the model")
            # else:
            #     methane = mdlutl.model.reactions.get_by_id("EX_cpd01024_e0")
            #     coef.update({methane.reverse_variable: ch4_weight, methane.forward_variable: -ch4_weight})
            
                
            ## defining the minimal probability objective
            output[percentOptimum][modelID] = {}
            coef, flux_fitting_target = {}, {}
            for rxn in base_model.reactions:
                if "rxn" == rxn.id[0:3]:
                    prob = reaction_probabilities[rxn.id]
                    currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
                    coef.update({currRxn.forward_variable: max(min_prob, (1 - float(prob) ** prob_exp))})
                    coef.update({currRxn.reverse_variable: max(min_prob, (1 - float(prob) ** prob_exp))})
                elif "EX_" == rxn.id[0:3]:
                    currRxn = mdlutl.model.reactions.get_by_id(rxn.id)
                    if rxn.id[3:11] in metabolomics_data:
                        flux_fitting_target[rxn.id] = -1*output[modelID]["max_growth"]*metabolomics_data[rxn.id[3:11]][modelID]
                    coef.update({currRxn.forward_variable: ex_weight})
                    coef.update({currRxn.reverse_variable: ex_weight})

            ## Adjusting rxn bounds according to the maximum community growth and metabolomics data
            ## This works around Linear Constraint problems and Quadratic Constraint infeasiblity
            ### TODO replace constraining all metabolomics metabolites with only top (+)-correlating compounds, and their compount categories
            stuck_reactions = {}

            ## assigning the minimum objective function
            mdlutl.model.objective = mdlutl.model.problem.Objective(Zero, direction="min")
            mdlutl.model.objective.set_linear_coefficients(coef)

            ## simulating and checking the solution
            solution = mdlutl.model.optimize()
            if solution.status != "optimal":
                print(f"{model_path} is infeasible at {percentOptimum} of optimum growth")
                continue
                # while solution.status != "optimal":
                #     util.save("objConstPkg_contents", mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"])
                #     print(dir(mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"]),
                #           mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"])
                #     print("Infeasible: reducing objective constraint",0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb)
                #     mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb = 0.9*mdlutl.pkgmgr.getpkg("ObjConstPkg").constraints["objc"].lb
                #     solution = mdlutl.model.optimize()

            ## storing and exporting successful simulation results
            export_path = util.output_dir+"/LPs/"+modelID+"-final.lp"
            if not os.path.exists(export_path):
                os.makedirs(export_path)
            with open(export_path, "w") as out:
                out.write(str(mdlutl.model.solver))

            output[percentOptimum][modelID]["minimum probability objective"] = solution.objective_value
            records[3][modelID] = output[percentOptimum][modelID]["minimum probability objective"]
            output[percentOptimum][modelID]["solution"] = {}
            for rxn in mdlutl.model.reactions:
                if rxn.id not in rxn_record_hash:
                    rxn_record_hash[rxn.id] = {"id":rxn.id}
                    for mdlID in modelIDs:
                        rxn_record_hash[rxn.id][mdlID] = None
                    records.append(rxn_record_hash[rxn.id])
                rxn_record_hash[rxn.id][modelID] = solution.fluxes[rxn.id]
                output[percentOptimum][modelID]["solution"][rxn.id] = solution.fluxes[rxn.id]
            
            baseName = dirName+f"/{modelID}"
            with open(baseName+".json", "w") as out:
                dump(output, out, indent=3)
            df = DataFrame.from_records(records)
            df.to_csv(f"{baseName}.csv")
        
        with open(dirName+f"/{modelID}_output.json", "w") as out:
            dump(output[percentOptimum], out, indent=3)
        with open(dirName+f"/{modelID}_rxn_record_hash.json", "w") as out:
            dump(rxn_record_hash, out, indent=3)
        util.save(f"{percentOptimum}_{modelID}_records", records)

# import stored data
metabolites = util.load("metabolites")
metabolomics_path = "/scratch/shared/code/MicrobiomeNotebooks/Cliff/data/Cliff_Sample_Metadata_BGC_NMR.csv"
metabolomics_data = read_csv(metabolomics_path).drop(["New_index", "Unnamed: 0"], axis=1).set_index("Sample").to_dict()
feature_probabilities = util.load("feature_probabilities")
# media = util.msrecon.get_media("KBaseMedia/Complete")
media = util.msrecon.get_media("188065/1875/1")
print(media)


# R1_B_D2 / R1_B_D1 / R1_C_D2 are really slow for some reason

from glob import glob
from cobra.io import load_json_model
from json import load, dump
with open("/scratch/shared/code/MicrobiomeNotebooks/Cliff/data/dipeptide_exchanges.json", 'r') as jsonIn:
    dipeptide_exchanges = load(jsonIn)
    
missing_models = {"100": ["R2A_A_D1", "R1_B_D1", "R1_A_D1", "R1_B_D2"],
                  '90': ['R2A_A_D1', 'R1_A_D1', 'R1_B_D1', 'R1_B_D2', 'R2_A_D1', 'R2_C_D2'],
                  '80': ['R2A_A_D1', 'R1_A_D1', 'R1_A_D2', 'R1_B_D1', 'R1_B_D2', 'R1_C_D2', 'R2_A_D1', 'R2_C_D1', 'R2_C_D2'],
                  '70': ['R2A_A_D1', 'R2A_A_D2', 'R2A_C_D2', 'R1_A_D1', 'R1_A_D2', 'R1_B_D1', 'R1_B_D2', 'R1_C_D2', 'R2_A_D1', 'R2_B_D2', 'R2_C_D1', 'R2_C_D2'],
                  '60': ['R2A_A_D1', 'R2A_A_D2', 'R2A_C_D2', 'R1_A_D1', 'R1_A_D2', 'R1_B_D1', 'R1_B_D2', 'R1_C_D1', 'R1_C_D2', 'R2_A_D1', 'R2_B_D2', 'R2_C_D1', 'R2_C_D2'],
                  '50': ['R2A_A_D1', 'R2A_A_D2', 'R2A_C_D2', 'R1_A_D1', 'R1_A_D2', 'R1_B_D1', 'R1_B_D2', 'R1_C_D1', 'R1_C_D2', 'R2_A_D1', 'R2_B_D1', 'R2_B_D2', 'R2_C_D1', 'R2_C_D2']}
# models = list(glob("models/*.json")) # ["models/R2A_B_D2.json"] # ["models/R2A_A_D1.json", "models/R1_B_D2.json"] #  # 
from itertools import chain
models = [f"models/{m}.json" for m in list(chain.from_iterable(list(missing_models.values())))]
parallelize = True
commKin = 750
cUptake = 500
if parallelize:
    from multiprocess import Pool
    from os import cpu_count
    from numpy import array_split
    
    # a thread is given for each community model
    pool_count = min(cpu_count(), len(models)*3)
    print(f"{pool_count} threads will be created")
    pool = Pool(pool_count)
    args = [[[model_path], feature_probabilities, metabolomics_data, metabolites,
             dipeptide_exchanges, media, commKin, cUptake, missing_models]
            for model_path in models]
    list_of_outputs = pool.map(runCommModel, args)
else:
    outputs = runCommModel([models, feature_probabilities, metabolomics_data, metabolites,
                            dipeptide_exchanges, media, commKin, cUptake, missing_models])

python version 3.11.1
KBBaseModules 0.0.1
modelseedpy 0.3.3
cobrakbase 0.4.0
Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/
174 threads will be created


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)





ForkPoolWorker-1 models/R2A_A_D1.json

1726247678.346249 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)





ForkPoolWorker-2 models/R1_B_D1.json

...

30 Species30


1726248483.7235684 INFO: Making atp hydrolysis reaction for species: Species31



Species22 2220 Species20


1726248484.8380296 INFO: Making atp hydrolysis reaction for species: Species21
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




ForkPoolWorker-49 models/R1_A_D2.json
1 missing R1_A_D2
31 Species31
ForkPoolWorker-49 models/R1_A_D2.json


## combine the various outputs

In [25]:
# %run cliffcommutil.py

# from pandas import read_csv, concat
# from glob import glob
# from os import remove

# CSVs = []

#     for csv in glob(util.output_dir+"/*%.csv"):
#         df = read_csv(csv)
#         # print(df["id"])
#         remove(csv)
#         CSVs.append(df)

# totCSV = concat(CSVs).drop("Unnamed: 0", axis=1).set_index("id").groupby(level=0).sum()
# display(totCSV)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir


,Unnamed: 0.1,SF2_A_D1,SF2_A_D2,SF2_B_D1,SF2_B_D2,SF2_C_D1,R2A_A_D2,SF2_C_D2,R2A_B_D1,R2A_B_D2,...,R2_C_D2,R2_A_D2,R2_C_D1,R2_B_D2,R1_C_D2,R1_B_D1,R1_C_D1,R1_A_D2,R1_B_D2,R1_A_D1
id,,,,,,,,,,,,,,,,,,,,,
EX_cpd00001_e0,12619818,0.0,0.0,55380.981518,54237.515140,42741.332130,0.0,14765.727163,0.0,0.0,...,0.0,5254.596150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_cpd00007_e0,12591144,0.0,0.0,-32742.123623,-32039.059057,-25623.848739,0.0,-10593.837628,0.0,0.0,...,0.0,-2430.128847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_cpd00009_e0,12575424,0.0,0.0,-6932.646960,-6783.988908,-10612.005906,0.0,-1056.885681,0.0,0.0,...,0.0,-97.456634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_cpd00011_e0,12585912,0.0,0.0,23290.527786,22783.077453,15577.560878,0.0,10695.010054,0.0,0.0,...,0.0,818.023950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EX_cpd00012_e0,12757674,0.0,0.0,2785.271916,2725.546807,4727.964647,0.0,284.749576,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sul00010_c3,536346,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sul00010_c4,522888,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sul00010_c5,1067892,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## combine the simulation outputs

In [2]:
%run cliffcommutil.py

from json import load, dump
from glob import glob
from os import remove

# combine outputs
outputs = {}
maxGrowthPercent = 100
baseDir = util.output_dir+f"/{maxGrowthPercent}"
for output in glob(baseDir+"/*_output.json"):
    mag = output.replace(baseDir+"/", '').replace("_output.json", '')
    with open(output, 'r') as jsonIn:
        outputs.update(load(jsonIn))
    # remove(output)
        
with open(baseDir+"/MAGfluxes.json", 'w') as jsonOut:
    dump(outputs, jsonOut, indent=3)
    
# combine rxn_record_hashes
outputs = {}
for output in glob(baseDir+"/*_rxn_record_hash.json"):
    mag = output.replace(baseDir+"/", '').replace("_rxn_record_hash", '')
    with open(output, 'r') as jsonIn:
        outputs.update({mag: load(jsonIn)})
    # remove(output)
        
with open(baseDir+"/MAG_rxn_record_hash.json", 'w') as jsonOut:
    dump(outputs, jsonOut, indent=3)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


In [16]:
print(len(set([i for i in totCSV.index])) == len([i for i in totCSV.index]))

True


In [2]:
import subprocess

# Run the command and capture the output
result = subprocess.run(["python", "-V"], capture_output=True, text=True)

# Print the output
print(result.stdout)

Python 3.11.1



## check the outputs

In [8]:
%run cliffcommutil.py

from json import load, dump
from glob import glob
from os import remove

output = util.output_dir+"/R1_A_D1/0.7%.json"
with open(output, 'r') as jsonIn:
    data = load(jsonIn)
    
print(len(data["R1_A_D1"]["solution"]), len([v for k,v in data["R1_A_D1"]["solution"].items() if v != 0]))

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
22294 2661


In [ ]:
from glob import glob
from itertools import product

sites = ["R2A", "SF2", "R1", "R2"]
cores = ["A", "B", "C"]
depths = ["D1", "D2"]

sampleParts = product(sites, cores, depths)
print(list(sampleParts))
samples = [f"{site}_{core}_{depth}" for (site, core, depth) in product(sites, cores, depths)]

for x in ["100", "90.0", "80.0", "70.0", "60.0", "50.0"]:
    models = glob(f"/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput/{x}/*_output.json")
    sampleModels = [model.split("/")[-1].replace("_output.json", '') for model in models]
    if set(samples) - set(sampleModels) != set():
        print(x, len(models))

# computing the community interactions

In [3]:
%run cliffcommutil.py

from json import load
maxGrowthPercent = 100
baseDir = util.output_dir+f"/{maxGrowthPercent}"
with open(f"/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput/{maxGrowthPercent}/MAGfluxes.json", 'r') as jsonIn:
    CommunityFluxSolution = load(jsonIn)
# CommunityFluxSolution = {k2:v2 for k,v in CommunityFluxSolution.items() for k2, v2 in v.items()}   # temporary patch of combining bug

from cobra.io import load_json_model
from pandas import read_csv
metabolomics_path = "/scratch/shared/code/MicrobiomeNotebooks/Cliff/data/Cliff_Sample_Metadata_BGC_NMR.csv"
metabolomics_data = read_csv(metabolomics_path)
mag_abundances = util.load("refined_MAG_abundances")
metabolites = util.load("metabolites")
metabolite_names = util.load("metabolite_names")
mag_list = util.load("mag_list")

sample_interval_exchanges = {}#[Sample interval][metabolite] = Flux
mag_sample_flux_profiles = {}#[Sample][MAG][Reaction] = Flux
rxn_data_hash = {}
mag_biomass = {}
mag_rescaled_biomass = {}
biomasses = {}
for sample, content in CommunityFluxSolution.items():
    sample_interval_exchanges[sample] = {}
    mag_sample_flux_profiles[sample] = {}
    rxn_data_hash[sample], mag_biomass[sample] = {}, {}
    base_model = load_json_model("models/"+sample+'.json')
    for rxnobj in base_model.reactions:
        if rxnobj.id[0:3] == "rxn":
            (baseid, compartment) = rxnobj.id.split("_")
            if baseid not in rxn_data_hash[sample]:
                rxn_data_hash[sample][baseid] = {
                    "id":baseid,
                    "equation":str(rxnobj),
                    "name":rxnobj.name,
                    "obj":rxnobj
                }
    print(sample, f"{len(rxn_data_hash[sample])} reactions", end="  ")
    biomasses[sample] = content["solution"]["bio1"]
    mag_biomass[sample] = {}
    mag_rescaled_biomass[sample] = {}
    for mag in mag_list:
        magname = mag[1].replace(".contigs__.RAST", '')
        # print(list(map(type, [sample, magname])))
        mag_biomass[sample][magname] = mag_abundances[sample][magname]*biomasses[sample]
        mag_rescaled_biomass[sample][magname] = 0 if magname not in mag_abundances[sample] else mag_abundances[sample][magname] 
    sample_mags = list(mag_abundances[sample].keys())
    mag_sample_flux_profiles[sample] = {}
    for rxn, flux in content["solution"].items():
        if rxn[0:3] == "EX_":
            sample_interval_exchanges[sample][rxn[3:11]] = content["solution"][rxn]
        elif rxn[0:3] == "rxn":
            (baseid, compartment) = rxn.split("_")
            if compartment[0:1] == "c":
                index = int(rxn.split("_")[1][1:])
                magname = sample_mags[index-1].replace(".contigs__.RAST", '') 
                ## store the flux profile for a mag in a given sample
                if magname not in mag_sample_flux_profiles[sample]:
                    mag_sample_flux_profiles[sample][magname] = {}
                total_flux = 0 if magname not in mag_abundances[sample] else flux*mag_abundances[sample][magname]
                mag_sample_flux_profiles[sample][magname][baseid] = total_flux
            # print(f"{len(mag_sample_flux_profiles[sample][magname])} fluxes")
    print(f"{len(mag_sample_flux_profiles[sample])} members")

#Computing metabolite interactions and printing dataframe of scaled mag flux profiles per sample inteval
record_hash = {}
record_dic = {}
records = []
indices = []
zero_biomass = {}
mag_metabolite_interactions = {}
for sample, modelFluxes in mag_sample_flux_profiles.items():
    mag_metabolite_interactions[sample] = {}
    zero_biomass[sample] = []
    record_dic[sample] = {}
    for magname, fluxes in modelFluxes.items():
        mag_metabolite_interactions[sample][magname] = {}
        for rxn, flux in fluxes.items():
            if rxn not in record_hash:
                record_hash[rxn] = {
                    "id":rxn,
                    "equation":rxn_data_hash[sample][rxn]["equation"],
                    "name":rxn_data_hash[sample][rxn]["name"]
                }
                records.append(record_hash[rxn])
            for met in rxn_data_hash[sample][rxn]["obj"].metabolites:
                if met.id.split("_")[1] == "e0" and met.id.split("_")[0] in metabolites:
                    if met.id not in mag_metabolite_interactions[sample][magname]:
                        mag_metabolite_interactions[sample][magname][met.id.split("_")[0]] = 0
                    if rxn_data_hash[sample][rxn]["obj"].metabolites[met] < 0:
                        #This transport is a consumption reaction; consumption should be negative
                        mag_metabolite_interactions[sample][magname][met.id.split("_")[0]] += -1*flux
                    elif rxn_data_hash[sample][rxn]["obj"].metabolites[met] > 0:
                        #This transport is a excretion reaction; excretion should be positive
                        mag_metabolite_interactions[sample][magname][met.id.split("_")[0]] += flux       
            if mag_biomass[sample][magname] == 0:
                zero_biomass[sample].append(magname)
                # continue
            record_hash[rxn][magname+"_"+sample] = flux #/mag_biomass[sample][magname]
            record_dic[sample][rxn] = flux #/mag_biomass[sample][magname]
            if rxn not in indices:
                indices.append(rxn)

print(records[:3])
MAGdf = pd.DataFrame.from_records(records).T #, columns=list(mag_sample_flux_profiles.keys())).T
# MAGdf = pd.DataFrame(record_dic) #, index=indices)
# df.
MAGdf.to_csv(util.output_dir+"/magFluxes.csv")

print(f"The non-growing biomasses", zero_biomass)
    
#Printing mag metabolite interation matrix
record_hash = {}
records = []
for magname, modelMets in mag_metabolite_interactions.items():
    new_record = {"MAG":magname}
    for sample, metInteractions in modelMets.items():
        new_record.update({met+"_"+sample: flux for met, flux in metInteractions.items()})
    for met in metabolites:
        total = count = 0
        for sample, metInteractions in modelMets.items():
            if met in metInteractions:
                total += metInteractions[met]
                count += 1
        if count > 0:
            new_record[met+"_ave"] = total/count
    records.append(new_record)
df = pd.DataFrame.from_records(records)
df.to_csv(baseDir+"/magMetaboliteInteractions.csv",index=False)

#Printing dataframe of Sample metabolite interactions per interval and per sample
# record_hash = {}
# records = []
# for sample in sample_interval_exchanges:
#     new_record[sample] = {}
#     for metabolite in sample_interval_exchanges[sample]:
#         new_record[metabolite] = sample_interval_exchanges[sample][metabolite]
#     records.append(new_record)
# df = pd.DataFrame(records)
# df.to_csv(util.output_dir+"/SampleIntervalMetaboliteInterations.csv",index=False)

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir
ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


1725556544.6722653 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


SF2_A_D1 2362 reactions  11 members


1725556561.6754322 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


SF2_B_D1 2516 reactions  15 members


...

1725557351.0636017 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


R2_B_D2 2924 reactions  32 members


1725557416.155094 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


R1_C_D2 2884 reactions  32 members
[{'id': 'rxn02201', 'equation': 'rxn02201_c1: cpd00443_c1 + cpd02920_c1 --> cpd00012_c1 + cpd00067_c1 + cpd00683_c1', 'name': '2-amino-4-hydroxy-6-hydroxymethyl-7,8-dihydropteridine-diphosphate:4-aminobenzoate 2-amino-4-hydroxydihydropteridine-6-methenyltransferase [c0]', 'Salt_Pond_MetaGSF2_A_D1_MG_DASTool_bins_concoct_out.23_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.11_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.13_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.28_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.33_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.40_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.55_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_D2_MG_DASTool_bins_concoct_out.8_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_H2O_MG_DASTool_bins_concoct_out.23_SF2_A_D1': 0.0, 'Salt_Pond_MetaGSF2_A_H2O_MG_DASTool_bins_co

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:
MAGdf.index

Index(['SF2_A_D1', 'SF2_A_D2', 'SF2_B_D1', 'SF2_B_D2', 'SF2_C_D1', 'R2A_A_D2',
       'R2A_B_D1', 'SF2_C_D2', 'R2A_B_D2', 'R2A_A_D1', 'R2A_C_D1', 'R2A_C_D2',
       'R2_A_D1', 'R2_B_D1', 'R2_C_D2', 'R2_A_D2', 'R2_C_D1', 'R2_B_D2',
       'R1_C_D2', 'R1_B_D1', 'R1_C_D1', 'R1_A_D2', 'R1_B_D2', 'R1_A_D1'],
      dtype='object')

# creating a COBRApy Interaction Model of each community and member fluxes

In [3]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase")

In [4]:
%run cliffcommutil.py
from cobra import Model, Reaction, Metabolite, io
from pandas import read_csv


def isNumber(val):
    import re
    remainder = re.sub("[0-9.-]+", "", str(val))
    if remainder != "": 
        # print(remainder)
        return False
    return True

# construct the interaction models for each sample community
sampleMAGs = util.load("refined_MAG_abundances")
maxGrowthPercent = 100
baseDir = util.output_dir+f"/{maxGrowthPercent}"
# display(read_csv(open(util.output_dir+"/magMetaboliteInteractions.csv")))
MAGdf = read_csv(open(baseDir+"/magMetaboliteInteractions.csv")).set_index("MAG")
# display(MAGdf)
interactionModel = Model()
magMets = {}
for sample, row in MAGdf.iterrows():
    ## add all of the interactions to the sample model
    mags = list(sampleMAGs[sample].keys())
    magMets[sample] = {}
    for metMAG, flux in row.items():
        met, mag = metMAG.split("_", 1)
        if not isNumber(flux):
            # print(mag, met, flux)
            continue
        if mag in mags:
            magMets[sample][mag] = magMets[sample].get(mag, {"rxn": Reaction(f"{sample}__{mag}", name=""), "mets": {}})
            metObj = Metabolite(met, name=msdb.compounds.get_by_id(met).name) #, compartment='c')   msdb.compounds.get(met)["name"]
            magMets[sample][mag]["mets"][metObj] = flux
    # for mag, metFluxes in magMets.items():
    for mag, content in magMets[sample].items():
        content["rxn"].add_metabolites(content["mets"])

    ## add reactions to the Interaction model
    interactionModel.add_reactions([content["rxn"] for content in magMets[sample].values()])

## write the model for visualizing interactions
io.save_json_model(interactionModel, baseDir+"/MAGInteractionModel.json")

Output files printed to:/scratch/shared/code/MicrobiomeNotebooks/Cliff/nboutput when using KBDevUtils.output_dir


1725557487.1637022 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


ModelSEED: /scratch/shared//sdkmount/run_local/workdir/tmp/


NameError: name 'msdb' is not defined

# Correlate methane flux and methane abundance of all communities

## there should be variability among the communities

# Test whether the models can produce methane given methane sources (glycine betaine, MPP, formate)